<a href="https://colab.research.google.com/github/pantipolo12/Diet-Recommendation-LLM/blob/main/IR_PROJECT_Diet_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OCR (Optical Character Recognition)

In [1]:
# @title
# Given Dataset (Chose relevant prescriptions)
!gdown 1DvcXqFzEDlrFQTOQV_qEZjDWBT1MnnFl
!gdown 1ENdiOHRYL8ImpFuBR9nLSRohF8nBmvJS
!gdown 1rHIa19GiUb5xlG6ptdJXyaTs5GD8tyl0
!gdown 196Ww4pmzDvRHNIhBU4zhwfO9cY7qQJI0

Downloading...
From: https://drive.google.com/uc?id=1DvcXqFzEDlrFQTOQV_qEZjDWBT1MnnFl
To: /content/prescription 5.pdf
100% 143k/143k [00:00<00:00, 10.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ENdiOHRYL8ImpFuBR9nLSRohF8nBmvJS
To: /content/prescription 6.pdf
100% 296k/296k [00:00<00:00, 46.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rHIa19GiUb5xlG6ptdJXyaTs5GD8tyl0
To: /content/prescription 7.pdf
100% 255k/255k [00:00<00:00, 93.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=196Ww4pmzDvRHNIhBU4zhwfO9cY7qQJI0
To: /content/prescription 8.pdf
100% 338k/338k [00:00<00:00, 41.7MB/s]


In [2]:
# Installing OCR libraries
%%capture
!pip install pypdfium2
!pip install poppler-utils
!pip install pdf2image
!pip install tesseract-ocr
!pip install libtesseract-dev
!pip install pytesseract

In [3]:
# Importing necessary libraries
import pypdfium2 as pyp
from pdf2image import convert_from_path
from IPython.display import display, Image
import cv2
import pytesseract
import os
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
# Removes any skew in the image
def deskew(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)
    coords = np.column_stack(np.where(gray > 0))
    angle = cv2.minAreaRect(coords)[-1]

    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    return rotated

# Extracts text from image
def extract_text_from_image(image):
    text = pytesseract.image_to_string(image)
    return text

In [5]:
# Directory where the pdfs are stored
data_dir = '/content'

In [6]:
# Returns tokenized text extracted from pdf or images after new line/empty string removal
def read_prescription(path):
  pdf = pyp.PdfDocument(path)
  pages = len(pdf)
  text = []

  for i in range(pages):
      raw = pdf[i].get_textpage().get_text_range()
      words = word_tokenize(raw)
      text.extend(words)

  # If the pdf is not read, that means we have an image and we read via OCR
  if len(text) == 0:
    pages = convert_from_path(path)
    extracted_text = []

    for page in pages:
        preprocessed_image = deskew(np.array(page))
        text1 = extract_text_from_image(preprocessed_image)
        extracted_text.append(text1)

    # The text extracted is erronous, we combine all of it and then split
    # via spaces and remove unnecessary new line characters and empty strings
    text.extend(extracted_text)
    temp = ""
    for x in text:
      temp += x + " "
    text = [temp]

    text = text[0].split(" ")
    remove = []
    for i in range(len(text)):
      text[i] = re.sub('\n', '', text[i])
      if text[i] == "":
        remove.append(i)
    for i in remove[::-1]:
      text.pop(i)

  return text

In [7]:
# Find all the pdf files in the current directory
pdf_files = [f for f in os.listdir(data_dir) if f.endswith('.pdf')]

# Empty df
df = pd.DataFrame(columns=['file', 'text'])
title = []
texts = []
for pdf_file in pdf_files:
  path = os.path.join(data_dir, pdf_file)
  text = read_prescription(path)
  texts.append(text)
  title.append(pdf_file)

df['file'] = title
df['text'] = texts

In [8]:
# Extracting medications or advice
keywords = ["medication", "medicine", "advice", "advised" , "food", "diet", "meal"]
medicines = []
for i in range(df.shape[0]):
  prescription = df.iloc[i][1]
  flag = False
  for word in keywords:
    for med in range(len(prescription)):
      if word in prescription[med].lower():
        prescription = prescription[med:]
        flag = True
      if flag:
        break
    if flag:
      break
  medicines.append(" ".join(prescription))

df["extracted"] = medicines

/tmp/ipython-input-1641984314.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prescription = df.iloc[i][1]


In [9]:
complete = []
for i in range(df.shape[0]):
  prescription = df.iloc[i][1]
  prescription = " ".join(prescription)
  complete.append(prescription)

df["complete"] = complete

/tmp/ipython-input-608721989.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prescription = df.iloc[i][1]


In [10]:
df

,file,text,extracted,complete
0,prescription 6.pdf,"[Bill, No, ., C.R, ., No, ., Name, Date, Remar...",Advice - Wt ( Kgs ) 82.2 kg . Small frequent m...,Bill No . C.R . No . Name Date Remarks - SITAR...
1,prescription 8.pdf,"[Firefox, Dr., VINEET, TALWARVARUNIARPIT, ., (...",ADVICE : CEMRI SPINE WITH PELVIS & SCREENING O...,Firefox Dr. VINEET TALWARVARUNIARPIT . ( Medic...
2,prescription 5.pdf,"[Name, CR, ., No, ., Phvsician, Dietician, Dia...",Medication : Inj Fiasp 4 units SC5 min before ...,Name CR . No . Phvsician Dietician Diagnosis T...
3,prescription 7.pdf,"[of, 1, Firefox, Dr., VINEET, TALWARVARUNIARPI...",ADVICE : PANOPTIC NGS PANEL TAB TAB ABIRATERON...,of 1 Firefox Dr. VINEET TALWARVARUNIARPIT . ( ...


In [11]:
df.iloc[1][2]

/tmp/ipython-input-65333983.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.iloc[1][2]


'ADVICE : CEMRI SPINE WITH PELVIS & SCREENING OF WHOLE SPINE REVIEVW WITH BIOPSY BLOCKS DTPA SCAN http : //rgciparas-clinical . local.rgcirc.net/doctor_dashboard/print cli .. 2D ECHO/ ECG TAB . DOLO 650MG -TDS TAB TYDOLO 50MG TDS TAB . GABAPIN 100MG -- HS Created At : 09-04-2024 01:50 pm Clinical Addendum Notes : Created At : 09-04-2024 03:00 pm CBC , KFT , LFT , ECG , 2 DECHO MRI dorsolumbar spine with screening of whole spine , pelvis ( with correlation with PSMA PET ) Print Date : 09-04-2024 Created By : Dr. VARUN GOEL Const Created By : Dr. VARUN GOEL'

# Gemini (A Generative AI Model developed by Google)

In [12]:
%%capture
!pip install google-generativeai

In [13]:
#Gemini API: {YOUR_API_KEY_HERE}
import os
import google.generativeai as genai
from google.colab import userdata

gemini_key = userdata.get('secret')

# Configure google-generativeai library with the API key
genai.configure(api_key = gemini_key)

In [14]:
# Supported models for generateContent method
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models

In [15]:
# Simple Test for Gemini
from IPython.display import Markdown
model = genai.GenerativeModel('gemini-2.5-flash')

response = model.generate_content("Can you provide medical details of say DOLO tablets.")

Markdown(response.text) #display text as Markdown

Dolo 650 is a widely used brand name in India for tablets containing **Paracetamol** (also known as Acetaminophen in some countries). It is one of the most common over-the-counter (OTC) medications globally.

Here are the medical details of Paracetamol (Dolo 650):

---

**DOLO 650 (Paracetamol)**

*   **Active Ingredient:** Paracetamol (Acetaminophen)
*   **Strength:** 650 mg (other strengths like 500 mg are also common)
*   **Drug Class:** Analgesic (pain reliever) and Antipyretic (fever reducer). It is NOT an NSAID (Non-Steroidal Anti-inflammatory Drug).

**1. Mechanism of Action (How it Works):**
Paracetamol's exact mechanism isn't fully understood, but it's believed to primarily work in the central nervous system (brain and spinal cord).
*   **Analgesic Effect:** It inhibits the synthesis of prostaglandins (chemicals that promote pain and inflammation) in the brain, thereby increasing the pain threshold. It has weaker effects on peripheral prostaglandins, explaining its limited anti-inflammatory action.
*   **Antipyretic Effect:** It acts on the hypothalamus (the part of the brain that regulates body temperature) to produce antipyresis (fever reduction) by peripheral vasodilation (widening of blood vessels) and increased sweating, leading to heat loss.

**2. Indications (What it's Used For):**
Dolo 650 is used to relieve:
*   **Fever:** Reduces elevated body temperature associated with various conditions (e.g., common cold, flu, infections).
*   **Mild to Moderate Pain:**
    *   Headache (including tension headaches, migraines)
    *   Muscle aches (myalgia)
    *   Backache
    *   Menstrual pain (dysmenorrhea)
    *   Toothache
    *   Arthritis pain (osteoarthritis)
    *   Post-operative pain
    *   Pain associated with colds and flu

**3. Dosage (Adults):**
*   **Standard Adult Dose (for 650 mg strength):** Typically one tablet (650 mg) every 4-6 hours, as needed.
*   **Maximum Daily Dose:** Do NOT exceed 4000 mg (4 grams) in 24 hours for adults. This translates to a maximum of 6 tablets of Dolo 650 in 24 hours.
*   **Duration:** Should not be used for more than a few days for pain or fever without consulting a doctor.
*   **Children:** Dosing for children is weight-based and should always be done under the guidance of a pediatrician.

**4. Side Effects:**
Paracetamol is generally well-tolerated at recommended doses. However, side effects can occur, though they are rare:
*   **Common (Rarely Occur):**
    *   Nausea
    *   Stomach upset
    *   Rash
*   **Serious (Very Rare, but Important):**
    *   **Liver Damage (Hepatotoxicity):** This is the most serious side effect, primarily occurring with overdose or chronic high doses, especially if combined with alcohol or in individuals with pre-existing liver conditions. It can be severe and life-threatening.
    *   Allergic reactions (hypersensitivity): Skin rashes, itching, swelling of the face/tongue/throat (angioedema), severe skin reactions (Stevens-Johnson Syndrome, Toxic Epidermal Necrolysis – extremely rare but serious).
    *   Blood disorders: Thrombocytopenia (low platelets), leukopenia (low white blood cells), agranulocytosis (very low granulocytes).

**5. Contraindications (When Not to Use):**
*   Severe liver disease or active liver failure.
*   Known hypersensitivity or allergy to paracetamol.

**6. Precautions & Warnings:**
*   **Liver Disease:** Use with caution and reduced dosage in individuals with existing liver impairment.
*   **Alcohol:** Avoid or limit alcohol consumption while taking paracetamol, as it significantly increases the risk of liver damage.
*   **Other Paracetamol-Containing Products:** Be careful not to take other medications (e.g., cold and flu remedies) that also contain paracetamol to avoid accidental overdose. Check labels carefully.
*   **Kidney Disease:** Use with caution and potential dose adjustment in severe kidney impairment.
*   **Malnutrition:** Individuals who are malnourished may be at higher risk of liver toxicity.
*   **Pregnancy & Breastfeeding:** Paracetamol is generally considered safe at recommended doses during pregnancy and breastfeeding, but always consult a doctor before use. It's often the preferred pain reliever during pregnancy.
*   **Asthma:** Some asthmatics, particularly those with aspirin-exacerbated respiratory disease, may experience bronchospasm with paracetamol, though it's less common than with NSAIDs.

**7. Drug Interactions:**
*   **Warfarin (anticoagulant):** Prolonged regular use of paracetamol at high doses can enhance the anticoagulant effect of warfarin, increasing the risk of bleeding.
*   **Alcohol:** Increases the risk of liver toxicity.
*   **Certain Anti-epileptics (e.g., carbamazepine, phenytoin, phenobarbital):** Can increase the risk of liver damage with paracetamol.
*   **Metoclopramide and Domperidone:** Speed up the absorption of paracetamol.
*   **Cholestyramine:** Reduces the absorption of paracetamol.

**8. Overdose:**
Paracetamol overdose is a medical emergency and can be fatal due to severe liver damage.
*   **Symptoms:** In the initial hours, symptoms may be vague (nausea, vomiting, abdominal pain, loss of appetite, general malaise). Liver damage becomes apparent 24-72 hours later.
*   **Action:** If an overdose is suspected, seek immediate medical attention (go to an emergency room or call emergency services) even if you feel well. There is an antidote (N-acetylcysteine) that is most effective when given promptly.

---

**Important Disclaimer:** This information is for general knowledge and informational purposes only, and does not constitute medical advice. Always consult with a qualified healthcare professional (doctor or pharmacist) before taking any medication, especially for diagnosis, treatment, or specific dosage recommendations. Do not self-diagnose or self-medicate.

In [16]:
# Prescription Test for Gemini
from IPython.display import Markdown, display

model = genai.GenerativeModel('gemini-2.5-flash')

for i in range(df.shape[0]):

  print(f"Below Prescription: {df.iloc[i][0]}")

  response = model.generate_content(df.iloc[i][2] + ". Please identify medicines and their associated symptoms from this prescription, also identify associated diseases. Based on this give Food Recommendations.")

  display(Markdown(response.text))

/tmp/ipython-input-2041243721.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Below Prescription: {df.iloc[i][0]}")
/tmp/ipython-input-2041243721.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  response = model.generate_content(df.iloc[i][2] + ". Please identify medicines and their associated symptoms from this prescription, also identify associated diseases. Based on this give Food Recommendations.")


Below Prescription: prescription 6.pdf


This prescription is for a 34-year-old pregnant woman (gestational age approximately 30-32 weeks based on Fundal Height).

Here's a breakdown of the medicines, their associated symptoms/diseases, and subsequent food recommendations:

---

### Patient Details:
*   **Age/Sex:** 34 years / Female
*   **Weight:** 82.2 kg (Indicates being overweight for her height/gestation)
*   **Primary Condition:** Antenatal Progress (Pregnancy) at approximately 30-32 weeks.
*   **Blood Pressure:** 130/70 mmHg (Borderline elevated, requires monitoring during pregnancy).

---

### Identified Medicines & Associated Conditions/Symptoms:

1.  **Tab. VIT D3 60k every week**
    *   **Medicine:** Vitamin D3 (Cholecalciferol)
    *   **Associated Disease/Symptom:** Vitamin D deficiency. Vitamin D is crucial for bone health, calcium absorption, immune function, and plays a vital role in preventing complications during pregnancy for both mother and fetus. Supplementation suggests the patient either has a deficiency or is at high risk.

2.  **Tab. CALCIMAX once daily after breakfast**
    *   **Medicine:** CALCIMAX is typically a Calcium supplement, often combined with Vitamin D3 and other minerals like Magnesium.
    *   **Associated Disease/Symptom:** Calcium deficiency/Requirement during pregnancy. Calcium is essential for the development of the fetal skeleton and teeth, as well as for maintaining maternal bone density, muscle function, and nerve transmission. Pregnancy significantly increases calcium demands.

3.  **Tab. FUR XT once daily, two hours after lunch**
    *   **Medicine:** FUR XT typically refers to an Iron and Folic Acid supplement. "FUR" often indicates Ferrous (iron) and "XT" might imply extra strength or extended release, often including Folic Acid.
    *   **Associated Disease/Symptom:** Iron deficiency anemia and Folic acid deficiency.
        *   **Iron:** Essential for red blood cell production, preventing maternal anemia (common in pregnancy), and supporting fetal growth and development.
        *   **Folic Acid:** Crucial for preventing neural tube defects (serious birth defects of the brain and spine) in the fetus. It also plays a role in red blood cell formation.

4.  **Syp. DIGENE / MUCAINE gel 1-2 tsf SOS for heartburn / acidity**
    *   **Medicine:** Digene/Mucaine gel are antacids, often containing active ingredients like aluminum hydroxide, magnesium hydroxide, and simethicone (Digene) or oxetacaine (Mucaine) for pain relief.
    *   **Associated Symptom:** Heartburn / Acidity. These are very common during pregnancy due to hormonal changes relaxing the esophageal sphincter and the growing uterus putting pressure on the stomach, pushing stomach acid upwards.

---

### Other Important Directives & Their Implications:

*   **Wt (Kgs) 82.2 kg:** Indicates the patient is overweight. This increases the risk of gestational diabetes, pre-eclampsia, and other pregnancy complications. Dietary management is crucial.
*   **BP 130/70:** Borderline high blood pressure. While not on medication for it, diet and exercise are key for management and preventing it from escalating to gestational hypertension or pre-eclampsia.
*   **Small frequent meals:** Recommended to manage nausea, heartburn, and blood sugar levels, especially during pregnancy.
*   **No outside food:** Reduces risk of foodborne illnesses and helps control calorie, salt, and sugar intake.
*   **Avoid caffeine/alcohol/Smoking:** Standard healthy pregnancy advice for fetal well-being.
*   **High Fiber Diet:** Important for preventing constipation (common in pregnancy), managing blood sugar, and promoting satiety.
*   **Avoid simple carbohydrates and sugar:** Crucial for managing weight, blood sugar levels (preventing gestational diabetes), and overall energy.
*   **Walking for 45 mins daily / To exercise daily as per physiotherapist instructions:** Essential for weight management, blood pressure control, circulation, mood, and preparing the body for labor.
*   **Fetal movement tracking:** Important for monitoring fetal well-being in the third trimester.
*   **Report to labour room in case of pain abdomen / excessive vomiting / bleeding / excessive discharge:** Important warning signs of potential complications.

---

### Food Recommendations:

Based on the medicines, symptoms, and direct dietary advice, here are the food recommendations for this pregnant patient:

**I. General Dietary Practices (Crucial for Pregnancy & Weight Management):**

*   **Small, Frequent Meals:** Instead of 3 large meals, aim for 5-6 smaller meals and snacks throughout the day. This helps manage hunger, blood sugar, and heartburn.
*   **Home-Cooked Meals:** Prioritize cooking at home to control ingredients, reduce sodium, unhealthy fats, and sugar, and ensure food hygiene. Avoid processed foods, packaged snacks, and restaurant meals as much as possible.
*   **Hydration:** Drink plenty of water throughout the day (around 8-10 glasses) to prevent dehydration and constipation.
*   **Avoid:**
    *   **Alcohol & Smoking:** Absolutely abstain.
    *   **Caffeine:** Limit coffee, tea, and caffeinated sodas. Consult your doctor for safe limits, but generally, less is better during pregnancy.
    *   **Raw/Undercooked Foods:** Avoid raw meat, poultry, fish, seafood (sushi), raw eggs, unpasteurized dairy, and unwashed fruits/vegetables to prevent foodborne illnesses (e.g., Listeria, Toxoplasmosis).

**II. Foods to Emphasize (Addressing Supplements, Fiber, & Nutrient Needs):**

1.  **High Fiber Foods (For constipation, satiety, blood sugar control):**
    *   **Whole Grains:** Oats, brown rice, whole wheat bread, whole wheat pasta, quinoa, barley.
    *   **Legumes:** Lentils, beans (black beans, kidney beans, chickpeas), peas.
    *   **Fruits:** Berries, apples (with skin), pears (with skin), oranges, bananas.
    *   **Vegetables:** Broccoli, spinach, carrots, bell peppers, leafy greens (kale, collard greens).

2.  **Lean Protein (For growth, satiety, and energy):**
    *   Skinless chicken and turkey
    *   Lean cuts of red meat (limit to 2-3 times a week, ensuring well-cooked)
    *   Fish (low mercury options like salmon, mackerel, sardines, tilapia - well-cooked)
    *   Eggs
    *   Lentils, beans, chickpeas, tofu, paneer (cottage cheese)

3.  **Healthy Fats (In moderation for brain development and energy):**
    *   Avocado
    *   Nuts (almonds, walnuts) and seeds (chia, flax, pumpkin)
    *   Olive oil, canola oil

4.  **Calcium & Vitamin D Rich Foods (Supporting CALCIMAX & VIT D3):**
    *   **Dairy:** Pasteurized milk, yogurt, cheese (low-fat options where possible).
    *   **Fortified Foods:** Plant-based milks (almond, soy, oat milk) fortified with calcium and vitamin D, fortified cereals.
    *   **Leafy Greens:** Spinach, kale, collard greens (though absorption might be less than dairy).
    *   **Fatty Fish:** Salmon, mackerel (also provide Vitamin D).

5.  **Iron & Folic Acid Rich Foods (Supporting FUR XT):**
    *   **Iron (Haem Iron for better absorption):** Lean red meat, poultry, fish.
    *   **Iron (Non-Haem Iron):** Lentils, beans, spinach, fortified cereals, tofu.
        *   **Tip:** Pair non-haem iron sources with Vitamin C-rich foods (e.g., bell peppers, oranges, tomatoes) to enhance iron absorption.
    *   **Folic Acid:** Leafy green vegetables (spinach, kale), broccoli, asparagus, oranges, fortified cereals, legumes (beans, lentils).

**III. Foods to Limit or Avoid (Addressing Weight, BP, Blood Sugar, Heartburn):**

1.  **Simple Carbohydrates & Sugars:**
    *   **Completely avoid:** Sugary drinks (soda, fruit juices with added sugar), candies, cakes, cookies, pastries, white sugar.
    *   **Limit:** White bread, white rice, refined cereals. Opt for their whole-grain counterparts.

2.  **High Sodium Foods (For BP management):**
    *   Processed foods, canned soups, cured meats, pickles, fast food, salty snacks (chips, pretzels). Cook with less added salt.

3.  **Unhealthy Fats (For weight management and overall health):**
    *   Deep-fried foods, highly processed snacks, excessive butter or margarine, high-fat dairy products.

4.  **Foods that may trigger Heartburn/Acidity (Individual variation, observe your triggers):**
    *   Spicy foods
    *   Greasy/fried foods
    *   Acidic foods (citrus fruits, tomatoes and tomato-based products)
    *   Chocolate
    *   Peppermint
    *   Large meals (reinforces "small frequent meals" advice)

---

**Important Note:** This advice is based on the provided prescription. Always follow the specific instructions and advice of your treating doctor and dietitian. If you have any concerns or questions about your diet or medication, please consult your healthcare provider.

Below Prescription: prescription 8.pdf


This clinical note provides crucial information, strongly suggesting a specific medical condition and management plan.

**Disclaimer:** I am an AI and cannot provide medical advice. This information is for educational purposes only and should not replace consultation with a qualified healthcare professional. Always follow your doctor's instructions regarding medications, tests, and diet.

---

### 1. Identified Medicines & Associated Symptoms:

Here's a breakdown of the medications, their common uses, and the symptoms they are prescribed to alleviate:

*   **DOLO 650MG -TDS (Paracetamol/Acetaminophen 650mg, three times a day):**
    *   **Drug Class:** Analgesic (pain reliever), Antipyretic (fever reducer).
    *   **Associated Symptoms:** Pain (mild to moderate, e.g., muscle pain, joint pain, headache, back pain), fever.
    *   **Implication:** The patient is experiencing pain.

*   **TYDOLO 50MG TDS (Likely Tramadol 50mg, three times a day):**
    *   **Drug Class:** Opioid analgesic (weak opioid). Tramadol is a synthetic opioid that works on the central nervous system to relieve pain.
    *   **Associated Symptoms:** Moderate to moderately severe pain. Often used when simpler pain relievers like paracetamol are not sufficient.
    *   **Implication:** The patient is experiencing significant pain that requires a stronger pain reliever.

*   **GABAPIN 100MG -- HS (Gabapentin 100mg, at bedtime):**
    *   **Drug Class:** Anticonvulsant, Neuropathic pain agent. It works by affecting certain chemicals and nerves in the body that send pain signals.
    *   **Associated Symptoms:** Neuropathic pain (nerve pain), such as tingling, burning, numbness, or shooting pain. It's often given at bedtime due to potential drowsiness and to help manage night pain.
    *   **Implication:** The patient is experiencing nerve-related pain, which is often a component of pain from spinal issues or nerve compression.

**Summary of Symptoms based on medications:** The patient is experiencing **significant pain**, likely with a **neuropathic (nerve pain) component**, and potentially some inflammation.

---

### 2. Identified Associated Diseases/Conditions:

The combination of imaging, procedures, and medications strongly points towards a specific and serious condition:

*   **Imaging/Procedures:**
    *   **"CEMRI SPINE WITH PELVIS & SCREENING OF WHOLE SPINE REVIEVW WITH BIOPSY BLOCKS"**: This indicates a detailed investigation of the spine and pelvis for abnormalities, with the intent to take tissue samples (biopsy) for diagnosis. This is done to identify the nature of lesions (e.g., tumor, infection).
    *   **"PSMA PET"**: **This is the most significant clue.** PSMA (Prostate-Specific Membrane Antigen) PET (Positron Emission Tomography) scans are primarily used to detect and stage **prostate cancer**, especially when it has spread (metastasized) or recurred. The "correlation with PSMA PET" means the MRI findings are being compared with the PET scan results.
    *   **"DTPA SCAN"**: This is a kidney scan, often used to assess kidney function and drainage. In the context of cancer, it might be used to check for kidney involvement, assess kidney health before certain treatments, or monitor for potential obstruction.
    *   **"2D ECHO/ECG"**: These are tests to assess heart function. They are routinely performed before major procedures, certain cancer treatments (like some chemotherapies that can affect the heart), or if the patient has cardiac risk factors.

*   **Medications (as discussed above):** The aggressive pain management (Paracetamol + Tramadol + Gabapentin) indicates severe, likely chronic, and neuropathic pain, which is very common with bone metastases.

*   **Lab Tests (CBC, KFT, LFT):** These are routine blood tests to assess general health, check for anemia, infection, kidney function, and liver function. They are crucial for monitoring overall health and safe medication administration, especially in cancer patients.

**Conclusion on Associated Disease:**
Given the strong evidence from the PSMA PET scan and the need for spine/pelvis biopsies, the primary associated disease is very likely **Prostate Cancer, with suspected metastatic spread to the spine and pelvis.** The pain management is directed at the severe pain often associated with bone metastases and potential nerve compression.

---

### 3. Food Recommendations (Based on Prostate Cancer, Bone Metastases, and Medication Side Effects):

These recommendations are general and should be discussed with the treating physician or a registered dietitian. The goals are to support overall health, manage potential side effects, and promote an anti-inflammatory environment.

**A. General Nutritional Support for Cancer Patients:**

1.  **Focus on Whole, Unprocessed Foods:** Prioritize fruits, vegetables, whole grains, lean proteins, and healthy fats.
2.  **Maintain Adequate Calories and Protein:** To prevent weight loss and muscle wasting (cachexia), common in cancer.
    *   **Lean Proteins:** Chicken, fish, eggs, legumes (beans, lentils), tofu, low-fat dairy.
    *   **Healthy Fats:** Avocados, nuts, seeds, olive oil (provide energy and essential fatty acids).
3.  **Hydration is Key:**
    *   Drink plenty of water throughout the day (8-10 glasses) unless fluid restriction is advised. This is vital for kidney function (relevant with DTPA scan and medication clearance) and preventing constipation.

**B. Specific Recommendations for Prostate Cancer & Bone Health:**

1.  **Anti-Inflammatory Diet:** An anti-inflammatory approach can help manage pain and potentially slow cancer progression.
    *   **Fruits and Vegetables:** Aim for a wide variety of colorful fruits and vegetables (berries, leafy greens, broccoli, tomatoes, carrots, bell peppers). These are rich in antioxidants and phytochemicals.
    *   **Omega-3 Fatty Acids:** Found in fatty fish (salmon, mackerel, sardines), flaxseeds, chia seeds, walnuts. Omega-3s have anti-inflammatory properties.
    *   **Spices:** Turmeric (curcumin), ginger, garlic, and cayenne pepper have anti-inflammatory benefits.
2.  **Bone-Supporting Nutrients:** With bone metastases, supporting bone health is crucial, though the primary focus will be on treating the cancer.
    *   **Calcium:** Dairy products (milk, yogurt, cheese), fortified plant milks, leafy greens (kale, spinach), fortified cereals.
    *   **Vitamin D:** Fatty fish, fortified dairy/plant milks, sunlight exposure (discuss with doctor).
    *   **Magnesium:** Leafy greens, nuts, seeds, whole grains, legumes.
    *   **Vitamin K:** Leafy greens (kale, spinach), broccoli.
3.  **Lycopene-Rich Foods:** Tomatoes and tomato products (cooked tomatoes, tomato paste) are high in lycopene, an antioxidant that has been studied for its potential benefits in prostate health.

**C. Managing Medication Side Effects:**

1.  **Constipation (Common with Tramadol):**
    *   **Increase Fiber:** Whole grains (oats, brown rice), fruits with skin, vegetables, legumes, nuts, seeds. Introduce fiber gradually.
    *   **Adequate Fluids:** Drink plenty of water.
    *   **Regular Physical Activity (if allowed):** Even light walking can help bowel motility.
2.  **Nausea/Vomiting (Possible with Tramadol):**
    *   **Small, Frequent Meals:** Instead of 3 large meals, try 5-6 smaller meals.
    *   **Bland Foods:** Toast, crackers, rice, plain chicken, bananas, applesauce.
    *   **Avoid Strong Odors:** Foods with strong smells can trigger nausea.
    *   **Ginger:** Ginger tea, ginger ale (flat), or ginger chews can help soothe nausea.
    *   **Avoid Fatty, Greasy, or Spicy Foods:** These can worsen nausea.
3.  **Dizziness/Drowsiness (Tramadol, Gabapentin):**
    *   **Consistent Meal Times:** Avoid skipping meals to maintain blood sugar stability.
    *   **Stay Hydrated:** Dehydration can worsen dizziness.
4.  **Liver Support (Paracetamol is metabolized by the liver):**
    *   **Avoid Alcohol:** Alcohol adds stress to the liver and should be avoided, especially while on Paracetamol and other medications.
    *   **Cruciferous Vegetables:** Broccoli, cauliflower, Brussels sprouts contain compounds that support liver detoxification pathways.

**D. Foods to Limit or Avoid:**

*   **Highly Processed Foods:** Often high in unhealthy fats, sugar, and sodium, and low in nutrients.
*   **Excessive Red and Processed Meats:** Some studies suggest a link between high intake and prostate cancer risk/progression.
*   **Excessive Sugar and Refined Carbohydrates:** Can contribute to inflammation.
*   **Alcohol:** As mentioned, it's a liver toxin and can interact with medications.

**Always communicate any dietary changes or concerns with Dr. VARUN GOEL or your oncology team.** They can provide personalized advice based on your specific condition, treatment plan, and individual needs.

Below Prescription: prescription 5.pdf


Based on the prescription, here's an identification of the medicines, associated symptoms, diseases, and food recommendations:

---

### 1. Medicines and Associated Diseases:

The medications prescribed are both types of insulin:

1.  **Inj Fiasp (Insulin Aspart - rapid-acting insulin analog):**
    *   **Mechanism:** This is a very fast-acting insulin designed to be taken just before (5 minutes) a meal to cover the carbohydrate intake and prevent blood sugar spikes after eating.
    *   **Associated Disease:** **Diabetes Mellitus** (Type 1 or Type 2, requiring insulin therapy). Given the regular mealtime doses, this patient is dependent on insulin to manage her blood sugar, likely due to insufficient natural insulin production or significant insulin resistance that oral medications can no longer control.

2.  **Inj Levemir (Insulin Detemir - long-acting insulin analog):**
    *   **Mechanism:** This insulin provides a continuous, steady level of insulin throughout the day or night to control blood sugar levels between meals and overnight. It acts as a "basal" insulin.
    *   **Associated Disease:** **Diabetes Mellitus** (Type 1 or Type 2, requiring insulin therapy). Similar to Fiasp, its use confirms the diagnosis of diabetes needing insulin management.

**Primary Associated Disease:** The primary disease being managed with both Fiasp and Levemir is **Diabetes Mellitus**. Given the age (34 yrs/Female) and the insulin regimen (both basal and bolus insulin), it could be **Type 1 Diabetes** (where the body produces little to no insulin) or **Type 2 Diabetes** (where the body doesn't use insulin well, and has progressed to needing insulin therapy). The prescription details confirm the need for strict blood glucose control.

---

### 2. Associated Symptoms:

The symptoms associated with this treatment are primarily related to blood glucose levels:

1.  **Symptoms of the underlying disease (Diabetes Mellitus - if uncontrolled):**
    *   **Hyperglycemia (High Blood Sugar):** While the insulin aims to prevent this, if the doses are insufficient or the diet/exercise are not managed, symptoms can include increased thirst (polydipsia), frequent urination (polyuria), fatigue, blurred vision, slow-healing sores, and unexplained weight loss. The target readings (<95mg/dl fasting, <140mg/dl 1hr PP, <120mg/dl 2hr PP, <90mg/dl pre-meal) indicate a goal to avoid these symptoms.

2.  **Symptoms as a potential side effect of insulin therapy:**
    *   **Hypoglycemia (Low Blood Sugar):** This is the most significant acute side effect of insulin. If too much insulin is taken, or if a meal is missed/delayed, or if exercise is increased without adjusting insulin, blood sugar can drop too low. Symptoms can include:
        *   **Mild to Moderate:** Shakiness, dizziness, sweating, hunger, irritability, rapid heartbeat, headache, blurred vision, weakness, anxiety.
        *   **Severe:** Confusion, difficulty concentrating, slurred speech, loss of coordination, seizures, unconsciousness.
    *   **Weight Gain:** Insulin can sometimes cause weight gain because it helps the body absorb glucose, preventing it from being excreted.
    *   **Injection Site Reactions:** Redness, swelling, itching, or pain at the injection site (usually mild and temporary).
    *   **Lipodystrophy:** Changes in fatty tissue under the skin (either thickening or indentation) at injection sites if areas are not rotated.

---

### 3. Food Recommendations:

The food recommendations are crucial for managing Diabetes Mellitus, especially when on insulin. The prescription itself provides excellent guidance: "Increase vegetables with each meal" and "Avoid having something within 2 hrs of major meal."

Here's a comprehensive set of food recommendations:

**A. General Principles for Diabetes Management:**

1.  **Consistency is Key:** Try to eat meals and snacks at consistent times each day. This helps synchronize with your insulin doses (especially Fiasp before meals and Levemir's basal action).
2.  **Portion Control:** Be mindful of portion sizes, especially for carbohydrate-rich foods, to manage blood sugar levels effectively.
3.  **Balanced Meals:** Aim for a balance of complex carbohydrates, lean protein, healthy fats, and plenty of non-starchy vegetables at each meal.

**B. Specific Food Group Recommendations:**

1.  **Carbohydrates (Choose Wisely):**
    *   **Focus on Complex Carbohydrates:** Opt for whole grains (brown rice, whole wheat bread, oats, quinoa, whole wheat pasta), legumes (beans, lentils, chickpeas), and starchy vegetables (sweet potatoes in moderation). These are high in fiber, which slows down sugar absorption and helps prevent sharp blood sugar spikes.
    *   **Limit Refined Carbohydrates & Sugars:** Strictly avoid sugary drinks (sodas, fruit juices with added sugar), candies, pastries, white bread, white rice, and processed snacks. These cause rapid blood sugar spikes.
    *   **Fruit Intake:** Enjoy whole fruits in moderation. They contain natural sugars and fiber. Avoid fruit juices as they lack fiber and can spike blood sugar quickly.

2.  **Proteins (Lean & Healthy):**
    *   **Choose Lean Sources:** Include lean meats (skinless chicken, turkey), fish (especially fatty fish like salmon for omega-3s), eggs, tofu, tempeh, legumes, and low-fat dairy (yogurt, cottage cheese).
    *   **Benefits:** Protein helps with satiety, muscle maintenance, and has a minimal direct impact on blood sugar levels.

3.  **Healthy Fats (In Moderation):**
    *   **Good Sources:** Avocados, nuts, seeds (chia, flax, pumpkin), olive oil, and fatty fish.
    *   **Benefits:** Healthy fats are important for overall health and can help you feel full, reducing the urge to snack.
    *   **Limit:** Saturated and trans fats found in fried foods, processed snacks, and excessive red meat.

4.  **Vegetables (Per Prescription - "Increase Vegetables with Each Meal"):**
    *   **Non-Starchy Vegetables:** Emphasize leafy greens (spinach, kale), broccoli, cauliflower, bell peppers, cucumbers, tomatoes, zucchini, carrots, green beans.
    *   **Benefits:** These are low in calories and carbohydrates but rich in fiber, vitamins, and minerals. They add volume and nutrients without significantly impacting blood sugar. Make them the largest portion of your plate.

**C. Meal Timing and Structure (Per Prescription):**

1.  **"Avoid having something within 2 hrs of major meal":** This is a critical instruction. It means structured eating with distinct meal times and avoids constant grazing. This allows the Fiasp insulin to work effectively on the meal's carbohydrates without overlapping with other food intake, which could lead to insulin "stacking" and increased risk of hypoglycemia.
2.  **Consistent Meal Times:** Especially important with Fiasp (taken 5 minutes before meals) and Levemir. Try to eat your breakfast, lunch, and dinner at roughly the same times each day.
3.  **No Skipping Meals:** Skipping meals, particularly after taking Fiasp, can lead to dangerous hypoglycemia.

**D. Hydration:**

*   **Water is Best:** Drink plenty of water throughout the day. Avoid sugary drinks and excessive caffeine.

**E. Managing Hypoglycemia Risk:**

*   **Always Carry Fast-Acting Glucose:** Glucose tablets, hard candies, or a small box of fruit juice should always be on hand to treat low blood sugar quickly.
*   **Recognize Symptoms:** Be vigilant for the symptoms of hypoglycemia and know how to treat them immediately.
*   **Inform Others:** Ensure family, friends, and colleagues know you have diabetes and what to do in case of a severe hypoglycemic event.

**F. Importance of SMBG:**

*   The prescription emphasizes "Self monitoring of blood glucose (SMBG) as advised." This is crucial. The food choices you make will directly impact these readings. Regularly monitoring will help you understand how different foods affect your blood sugar and allow for necessary adjustments in diet and insulin (in consultation with your doctor).

**G. Professional Guidance:**

*   It is highly recommended that the patient also consult with a Registered Dietitian or a Certified Diabetes Educator (like Komal Sharma, who is listed) for personalized meal planning and guidance. They can help with carbohydrate counting, portion sizing, and making food choices that fit individual preferences and cultural norms while managing diabetes effectively.

Below Prescription: prescription 7.pdf


This appears to be a prescription for a patient undergoing treatment for **Prostate Cancer**, likely with bone metastases, given the combination of medications. Dr. Arpit Jain is a Medical Oncology consultant, which further supports this diagnosis.

Here's a breakdown of the medicines, associated symptoms (side effects), and food recommendations:

---

### **Important Disclaimer:**

I am an AI and cannot provide medical advice. This information is for educational purposes only and is based on common medical knowledge. **Always follow the specific instructions and advice of Dr. Arpit Jain or your healthcare team.** Do not make any changes to your diet or medication regimen without consulting your doctor.

---

### **1. Medicines Identified & Associated Disease:**

**Associated Disease:** Most likely **Metastatic Castration-Resistant Prostate Cancer (mCRPC)**, especially with bone involvement.

1.  **TAB ABIRATERONE 1 GM OD EMPTY STOMACH**
    *   **Medicine:** Abiraterone (brand names include Zytiga, Yonsa).
    *   **Class:** Androgen biosynthesis inhibitor.
    *   **Purpose:** Treats advanced prostate cancer by reducing the production of androgens (male hormones) throughout the body, including in tumors.
    *   **Note on "PANOPTIC NGS PANEL":** This refers to a **Next-Generation Sequencing (NGS) diagnostic panel**, likely to analyze the patient's tumor genetics for specific mutations that might guide further treatment decisions, rather than a medicine itself.

2.  **WYSOLONE 5 MG BD WITH MEAL**
    *   **Medicine:** Wysolone (brand name for **Prednisolone**).
    *   **Class:** Corticosteroid.
    *   **Purpose:** Often co-prescribed with Abiraterone to manage its side effects, particularly those related to mineralocorticoid excess (like hypertension and hypokalemia), and can also have anti-cancer effects.

3.  **INJ ZOLEDEC RAZ 1 OD 4 MG IN 100 ML NS IV OVER 30 MINS ON 15.05.24**
    *   **Medicine:** Zoledec Raz (likely a brand name for **Zoledronic Acid** or Zoledronate).
    *   **Class:** Bisphosphonate.
    *   **Purpose:** Used to prevent or treat bone complications (skeletal-related events like fractures, spinal cord compression) in patients with bone metastases, which is common in advanced prostate cancer. Also used to manage hypercalcemia of malignancy.

4.  **TAB CARCIUM 1 OD DAILY**
    *   **Medicine:** Carcium (likely a **Calcium supplement**).
    *   **Purpose:** Often prescribed alongside Zoledronic Acid to prevent hypocalcemia (low calcium levels) and support bone health. It's also often given with steroids (Prednisolone) to counteract their effect on bone density.

**Monitoring Tests Ordered:**
*   **BP MONITORING:** Blood Pressure monitoring.
*   **CBC:** Complete Blood Count (checks blood cells).
*   **KFT:** Kidney Function Test.
*   **LFT:** Liver Function Test.
*   **iCa2+:** Ionized Calcium (direct measure of biologically active calcium).
*   **VITAMIN D3:** Vitamin D level.
*   **ECG:** Electrocardiogram (checks heart electrical activity).
These tests are crucial to monitor treatment efficacy and potential side effects on various organ systems.

---

### **2. Associated Symptoms (Side Effects) of Medicines:**

*   **Abiraterone:**
    *   **Common:** Fatigue, joint pain/swelling, hot flashes, edema (swelling), hypertension (high blood pressure), hypokalemia (low potassium), elevated liver enzymes, diarrhea, urinary tract infection.
    *   **Less common but serious:** Adrenal insufficiency, cardiac issues (arrhythmias, heart failure), elevated blood glucose.
    *   *(Prednisolone is given concurrently to mitigate some of these side effects, especially hypertension and hypokalemia.)*

*   **Prednisolone (Wysolone):**
    *   **Common (even at low dose):** Increased appetite, indigestion/heartburn, mood changes (irritability, anxiety), sleep disturbances, fluid retention, increased blood sugar.
    *   **Long-term/higher dose risks:** Osteoporosis (bone thinning), adrenal suppression, weight gain, thinning skin, cataracts, glaucoma, increased infection risk, muscle weakness.

*   **Zoledronic Acid (Zoledec Raz):**
    *   **Common (especially after the first infusion):** Flu-like symptoms (fever, fatigue, bone/joint/muscle pain), headache.
    *   **Kidney effects:** Renal impairment (kidney damage) – hence KFT monitoring.
    *   **Calcium effects:** Hypocalcemia (low calcium) – hence iCa2+ monitoring and calcium/Vitamin D supplementation.
    *   **Rare but serious:** Osteonecrosis of the jaw (ONJ), atypical femoral fractures.

*   **Calcium Supplement (Carcium):**
    *   Generally well-tolerated.
    *   **Potential:** Constipation, bloating, gas.
    *   **High doses (unlikely with this prescription if monitored):** Hypercalcemia (headache, nausea, fatigue, kidney stones).

*   **General from Cancer & Treatment:** Fatigue, pain, appetite changes, weight loss/gain, nausea, emotional distress.

---

### **3. Food Recommendations:**

The food recommendations aim to support overall health, manage medication side effects, and optimize nutrient intake, especially given the cancer diagnosis and specific medications.

**A. General Recommendations for Cancer Patients:**

1.  **Nutrient-Dense Diet:** Focus on whole foods:
    *   **Lean Proteins:** Fish, poultry (skinless), beans, lentils, tofu, eggs. Helps maintain muscle mass and supports recovery.
    *   **Whole Grains:** Brown rice, whole wheat bread, oats, quinoa. Provide sustained energy and fiber.
    *   **Plenty of Fruits & Vegetables:** A wide variety of colors for vitamins, minerals, and antioxidants. Aim for 5-9 servings daily.
    *   **Healthy Fats:** Avocados, nuts, seeds, olive oil.

2.  **Hydration is Key:** Drink plenty of water throughout the day. This is crucial for kidney function (especially with Zoledronic Acid), managing constipation, and overall well-being. Aim for 8-10 glasses of water daily unless advised otherwise by your doctor.

3.  **Small, Frequent Meals:** If appetite is poor or nausea is an issue, eating smaller portions more often can be easier to tolerate than large meals.

**B. Specific Recommendations based on Medications & Side Effects:**

1.  **Abiraterone (Empty Stomach):**
    *   **Strictly adhere to "empty stomach" instruction:** Take Abiraterone at least 1 hour before or 2 hours after food. Plan your meals around this dosing schedule. This is crucial for proper absorption.

2.  **Prednisolone (Wysolone) - "WITH MEAL" & Side Effects:**
    *   **Take with food:** Always take Prednisolone with a meal or a snack to reduce stomach upset, indigestion, or heartburn.
    *   **Manage Increased Appetite/Weight Gain:**
        *   Focus on **high-fiber, low-calorie foods** to feel full without excessive calorie intake (e.g., lots of vegetables, lean proteins).
        *   Limit sugary drinks, processed snacks, and foods high in unhealthy fats.
    *   **Manage Blood Sugar (Hyperglycemia Risk):**
        *   Choose **complex carbohydrates** (whole grains) over refined sugars.
        *   Pair carbohydrates with protein and healthy fats to slow sugar absorption.
        *   Limit fruit juices and sugary beverages.
    *   **Manage Fluid Retention/Hypertension:**
        *   **Limit sodium (salt) intake:** Avoid processed foods, canned soups, cured meats, pickles, and salty snacks.
        *   Cook at home using fresh ingredients and herbs/spices for flavor instead of salt.
    *   **Bone Health:** Continue to include calcium and vitamin D-rich foods (see below), as steroids can affect bone density over time.

3.  **Zoledronic Acid & Calcium/Vitamin D Supplement:**
    *   **Calcium-Rich Foods:** Incorporate dairy products (milk, yogurt, cheese), fortified plant milks (almond, soy, oat), leafy green vegetables (kale, collard greens), broccoli, fortified cereals, and canned salmon/sardines with bones.
    *   **Vitamin D-Rich Foods:** Fatty fish (salmon, mackerel), fortified milk/cereals, eggs. Your doctor is also monitoring and supplementing Vitamin D, but dietary sources are helpful.

4.  **Managing Other Potential Side Effects:**
    *   **Constipation (from calcium or general treatment):** Increase fiber intake (whole grains, fruits, vegetables, legumes) and ensure adequate fluid intake.
    *   **Diarrhea (from Abiraterone):**
        *   Follow the BRAT diet (bananas, rice, applesauce, toast).
        *   Eat small, frequent meals.
        *   Avoid greasy, fatty, spicy, or high-fiber foods temporarily.
        *   Drink plenty of fluids to prevent dehydration.
    *   **Nausea:**
        *   Eat bland foods (crackers, plain toast, rice).
        *   Avoid strong-smelling foods.
        *   Ginger (ginger ale, ginger tea, ginger candy) can help.
        *   Stay hydrated with clear broths and sips of water.

---

**Always discuss your diet with your doctor, oncologist, or a registered dietitian who can provide personalized advice based on your specific condition, treatment plan, and individual needs.** They can help you create a meal plan that supports your health and manages any side effects effectively.

In [18]:
# Prescription Test for Gemini
from IPython.display import Markdown, display

model = genai.GenerativeModel('gemini-2.5-flash')

for i in range(df.shape[0]):

  print(f"Below Prescription: {df.iloc[i][0]}")

  response = model.generate_content(df.iloc[i][3] + ". Please identify medicines and their associated symptoms from this prescription, also identify associated diseases.  Based on this give Food Recommendations. Structure response in simple table format")

  display(Markdown(response.text))

/tmp/ipython-input-2123317052.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Below Prescription: {df.iloc[i][0]}")
/tmp/ipython-input-2123317052.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  response = model.generate_content(df.iloc[i][3] + ". Please identify medicines and their associated symptoms from this prescription, also identify associated diseases.  Based on this give Food Recommendations. Structure response in simple table format")


Below Prescription: prescription 6.pdf


Here is the requested information in a table format:

| Category                   | Item/Medicine             | Associated Symptoms (from prescription) | Associated Diseases/Conditions (from prescription) | Food/Lifestyle Recommendations (from prescription) |
| :------------------------- | :------------------------ | :-------------------------------------- | :------------------------------------------------- | :------------------------------------------------- |
| **Patient Details**        | Mrs Vinti Agarwal         | Wt (Kgs) 82.2 kg, BP 130/70, 31 Weeks Pregnant | Pregnancy (Antenatal Care), Gestational Weight Management, Borderline Gestational Hypertension |                                                    |
|                            |                           | Pain abdomen / excessive vomiting / bleeding / excessive discharge (warning signs) |                                                    |                                                    |
| **Medicines & Reason**     | VIT D3 60k (every week)   | -                                       | Pregnancy (general supplement for Vitamin D needs) | -                                                  |
|                            | Tab. CALCIMAX (once daily after breakfast) | -                                       | Pregnancy (general supplement for Calcium needs, bone health) | -                                                  |
|                            | Tab. FUR XT (once daily, two hours after lunch) | -                                       | Pregnancy (general supplement for Iron needs, to prevent anemia) | -                                                  |
|                            | Syp. DIGENE / MUCAINE gel (1-2 tsf SOS) | Heartburn, Acidity                      | Heartburn, Acidity (common during pregnancy)      | -                                                  |
| **General Advice / Diet**  | -                         | -                                       | Pregnancy, Digestion, Overall Health, Weight Management | Small frequent meals                               |
|                            |                           | -                                       | Pregnancy, Food Safety, Weight Management          | No outside food                                    |
|                            |                           | -                                       | Pregnancy (fetal health & maternal well-being)     | Avoid caffeine/ alcohol/ Smoking                   |
|                            |                           | -                                       | Pregnancy, Constipation prevention, Digestion, Weight Management | High Fiber Diet                                    |
|                            |                           | -                                       | Pregnancy, Gestational Diabetes prevention, Weight Management | Avoid simple carbohydrates and sugar             |
|                            |                           | -                                       | Pregnancy, Physical fitness, Weight Management     | Walking for 45 mins daily                          |
|                            |                           | -                                       | Pregnancy, Physical fitness, Preparation for labor | To exercise daily as per physiotherapist instructions |

**Summary of Associated Diseases/Conditions:**
The primary condition is **Pregnancy (31 weeks Antenatal Care)**. Other associated conditions/concerns include:
*   Gestational Weight Management (due to 82.2 kg weight)
*   Borderline Gestational Hypertension (BP 130/70)
*   Common Pregnancy Discomforts like Heartburn and Acidity.
*   Preventative care for potential Vitamin D and Iron deficiencies.

**Food Recommendations based on the prescription:**
1.  **Small frequent meals:** To help with digestion and possibly manage heartburn/acidity, and maintain steady energy.
2.  **No outside food:** For food safety and better control over ingredients and portion sizes, contributing to weight management.
3.  **High Fiber Diet:** To aid digestion, prevent constipation (common in pregnancy), and contribute to satiety.
4.  **Avoid simple carbohydrates and sugar:** To manage blood sugar levels, prevent excessive weight gain, and reduce the risk of gestational diabetes.
5.  **Avoid caffeine/alcohol/smoking:** Crucial for fetal health and overall pregnancy well-being.

Below Prescription: prescription 8.pdf


Here's a breakdown of the medicines, associated symptoms, diseases, and food recommendations based on the provided prescription:

**Medical Information Summary:**
Mr. Khyali Ram Agrawal, a 67-year-old male, is a known case of metastatic Prostatic Acinar Adenocarcinoma (Gleason's score 4+4=8, Group grade 4). He underwent TURP in Nov 2022. Recent PET CT shows residual prostate PSMA expression, lymph node involvement, and widespread bony metastases in the pelvis and spine. His PSA levels are significantly rising, indicating disease progression. He is presenting with severe pain (lower back, pelvic) and neurological symptoms (decreasing right leg muscle mass), likely due to the bone metastases, especially in the spine.

---

### Medications, Associated Symptoms, and Diseases

| Medicine            | Associated Symptoms (Likely being treated)                                   | Associated Diseases (Likely being treated)                                   |
| :------------------ | :--------------------------------------------------------------------------- | :--------------------------------------------------------------------------- |
| **TAB. DOLO 650MG** (Paracetamol/Acetaminophen) | Lower back ache, Pelvic pain                                                 | Cancer pain (mild to moderate), particularly bone pain from metastatic prostate cancer. |
| **TAB TYDOLO 50MG** (Tramadol)         | Lower back ache, Pelvic pain (indicating moderate to severe pain)            | Cancer pain (moderate to severe), particularly bone pain from metastatic prostate cancer. |
| **TAB. GABAPIN 100MG** (Gabapentin)    | Decreasing Right leg muscle mass (suggests nerve involvement/neuropathy) | Neuropathic pain, radiculopathy, or nerve compression syndrome related to spinal metastases from prostate cancer. |

---

### Food Recommendations

Given Mr. Agrawal's diagnosis of metastatic prostate cancer, bone metastases, and symptoms of pain and muscle loss, the dietary recommendations aim to support overall health, manage symptoms, and potentially slow disease progression, while considering his age.

*   **Focus on Whole, Unprocessed Foods:**
    *   **Fruits and Vegetables:** Emphasize a wide variety of colorful fruits and vegetables (berries, leafy greens, cruciferous vegetables like broccoli, cauliflower, cabbage) known for their antioxidant and anti-inflammatory properties.
    *   **Whole Grains:** Choose whole grains like oats, brown rice, quinoa, and whole wheat bread for fiber and sustained energy.
    *   **Lean Proteins:** Include sources like fish (especially fatty fish rich in Omega-3s like salmon, mackerel), poultry (skinless), legumes (beans, lentils), and tofu. This is crucial for maintaining muscle mass, especially given his leg muscle loss.
    *   **Healthy Fats:** Incorporate monounsaturated and polyunsaturated fats from sources like olive oil, avocados, nuts, and seeds (flaxseeds, chia seeds).

*   **Bone Health Support (Crucial due to bone metastases):**
    *   **Calcium-Rich Foods:** Dairy products (milk, yogurt, cheese), fortified plant-based milks, leafy green vegetables, and fortified cereals.
    *   **Vitamin D:** Exposure to sunlight (if advised by doctor), fatty fish, fortified foods. Vitamin D aids calcium absorption.
    *   **Magnesium and Phosphorus:** Found in nuts, seeds, whole grains, and leafy greens, these are also important for bone health.

*   **Anti-inflammatory Diet:**
    *   **Omega-3 Fatty Acids:** Rich sources like fatty fish (salmon, sardines), flaxseeds, chia seeds, and walnuts. These have anti-inflammatory properties.
    *   **Turmeric, Ginger, Garlic:** Incorporate these spices known for their anti-inflammatory and antioxidant benefits.

*   **Limit/Avoid:**
    *   **Processed Foods:** High in unhealthy fats, sugar, and sodium, which can contribute to inflammation and poor health.
    *   **Red and Processed Meats:** Some studies suggest a link between high consumption of red and processed meats and prostate cancer progression.
    *   **Excess Saturated and Trans Fats:** Found in fried foods, processed snacks, and some animal products.
    *   **Sugary Drinks and Added Sugars:** Contribute to inflammation and can lead to unhealthy weight gain.
    *   **Excessive Alcohol:** Can weaken the immune system and interfere with nutrient absorption.

*   **Hydration:**
    *   Drink plenty of water throughout the day to support bodily functions, kidney health, and help manage potential constipation which can be a side effect of pain medications.

*   **Considerations for Appetite/Nausea (If present):**
    *   Small, frequent meals instead of large ones.
    *   Nutrient-dense smoothies or shakes.
    *   Easily digestible foods.

**Important Note:** These are general food recommendations. It is crucial for Mr. Agrawal to consult with a registered dietitian or his oncology team for personalized dietary advice, especially considering his specific medical condition, treatments, and any potential side effects.

Below Prescription: prescription 5.pdf


Here's a breakdown of the medicines, associated diseases, symptoms, and general food recommendations based on the provided prescription, structured in a table format:

| Medicine                     | Type/Action                                          | Associated Disease                                                         | Associated Symptoms (Treated by / Potential Side Effect)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| Inj Levemir (Insulin Detemir) | Long-acting basal insulin. Provides steady background insulin control. | Gestational Diabetes Mellitus (GDM) | **Treated Symptoms (Hyperglycemia):** Increased thirst, frequent urination, fatigue, blurred vision. **Potential Side Effects (Hypoglycemia):** Shakiness, dizziness, sweating, hunger, confusion, irritability, headache. Weight gain. Injection site reactions. | *   **Consistent Bedtime Snack (if needed):** While the prescription advises "Avoid having something within 2 hrs of major meal," discuss with the dietician if a small, balanced bedtime snack is appropriate to prevent overnight hypoglycemia, especially with basal insulin. If no snack is advised, ensure dinner is well-balanced and eaten at a consistent time.
                               |                                                      |                                                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|                               |                                                      |                                                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | *   **Avoid Sugar-Sweetened Beverages:** This includes soft drinks, regular fruit juices, and many energy drinks, which can cause sharp blood sugar spikes.
*   **Processed Foods:** Limit highly processed foods as they often contain hidden sugars, unhealthy fats, and refined carbohydrates.

**Review with your Dietician:** It's essential to follow the specific advice of Komal Sharma, your dietician, as these are general guidelines and your personalized plan will be tailored to your precise needs, glucose readings, and the details of your pregnancy.

Below Prescription: prescription 7.pdf


Here's a breakdown of the medicines, their associated potential side effects, and the patient's diseases and symptoms, followed by food recommendations:

### Medicines and Potential Side Effects

| Medicine          | Potential Side Effects (Associated Symptoms)                                                                                                              |
| :---------------- | :-------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **ABIRATERONE**   | Fatigue, joint pain/swelling, hot flashes, high blood pressure, fluid retention (edema), hypokalemia (low potassium), liver enzyme elevation.            |
| **WYSOLONE**      | Increased appetite, fluid retention, weight gain, indigestion, mood changes, insomnia, increased blood sugar, bone density loss (with long-term use).   |
| **INJ ZOLEDEC**   | Flu-like symptoms (fever, body aches) especially after the first dose, nausea, vomiting, constipation or diarrhea, hypocalcemia (low calcium levels).     |
| **TAB CARCIUM**   | Constipation, upset stomach (if taken in excess or without enough fluid).                                                                                 |

*Note: PANOPTIC NGS PANEL is a diagnostic test, not a medicine, so it's not included in this table.*

### Associated Diseases

*   **Prostatic Acinar Adenocarcinoma (Prostate Cancer):** A type of cancer originating in the prostate gland.
*   **Metastatic Prostate Cancer:** The cancer has spread from the prostate to other parts of the body, specifically to a left perirectal lymph node and multiple bones in the pelvis and spine.

### Patient's Reported Symptoms (Current Complaints from the Prescription)

These symptoms are primarily associated with his underlying disease, particularly the metastatic spread.

*   Lower back ache since 1 month
*   Pelvic pain since 1 month
*   Decreasing Right leg muscle mass since 1 month
*   Urinary retention (in August 2022)
*   Pain in Right Hip (taking Ultracet occasionally for this)

---

### Food Recommendations

Given the diagnosis of metastatic prostate cancer, bone involvement, and the medications being taken (Abiraterone, Wysolone, Zoledronic Acid, Calcium), the following food recommendations are aimed at supporting overall health, managing potential side effects, and promoting well-being:

**1. Foods to Encourage:**

*   **Fruits and Vegetables:** Aim for a wide variety of colorful fruits and vegetables (at least 5 servings daily). They provide antioxidants, vitamins, and fiber.
    *   *Specifically for Prostate Health:* Cooked tomatoes (rich in lycopene), cruciferous vegetables like broccoli, cauliflower, cabbage, and kale.
*   **Whole Grains:** Opt for whole-grain bread, brown rice, oats, and quinoa over refined grains. They provide sustained energy and fiber.
*   **Lean Protein Sources:** Fish (especially fatty fish like salmon, mackerel, sardines for Omega-3s), poultry (skinless), beans, lentils, and tofu. This is important for maintaining muscle mass, especially with decreasing right leg muscle mass.
*   **Calcium-Rich Foods:** Despite taking supplements, dietary sources are beneficial. Include low-fat dairy products (milk, yogurt, cheese), fortified plant milks, dark leafy greens (collard greens, spinach), and fortified cereals.
*   **Vitamin D-Rich Foods:** Fatty fish, fortified milk/cereals. Sunlight exposure is also important, but dietary sources help.
*   **Healthy Fats:** Avocados, nuts, seeds, olive oil.
*   **Hydration:** Drink plenty of water throughout the day to help with kidney function, prevent constipation, and support overall health.

**2. Foods to Limit/Avoid:**

*   **High-Saturated and Trans Fats:** Limit red and processed meats, fried foods, and high-fat dairy. These may contribute to inflammation and could potentially influence cancer progression.
*   **Excess Sodium:** With medications like Abiraterone that can cause fluid retention and hypertension, limiting high-sodium processed foods, canned goods, and salty snacks is important.
*   **Simple Sugars and Refined Carbs:** Limit sugary drinks, desserts, and white bread. This helps manage blood sugar, especially since Wysolone can elevate blood glucose levels.
*   **Alcohol:** Moderate intake, as advised by your doctor. Alcohol can interact with medications and impact liver function.
*   **Grapefruit/Grapefruit Juice:** May interact with certain medications, including Abiraterone. It's best to avoid or discuss with your doctor.

**3. General Advice:**

*   **Smaller, More Frequent Meals:** This can help manage indigestion and maintain energy levels, especially if experiencing fatigue or appetite changes.
*   **Chew Thoroughly:** Aids digestion.
*   **Listen to Your Body:** Pay attention to how different foods make you feel and adjust your diet accordingly.
*   **Consult a Dietitian/Nutritionist:** For personalized advice, especially given the complex medical situation and medications, consulting a registered dietitian who specializes in oncology is highly recommended.

In [19]:
response.candidates[0].content.parts[0].text

"Here's a breakdown of the medicines, their associated potential side effects, and the patient's diseases and symptoms, followed by food recommendations:\n\n### Medicines and Potential Side Effects\n\n| Medicine          | Potential Side Effects (Associated Symptoms)                                                                                                              |\n| :---------------- | :-------------------------------------------------------------------------------------------------------------------------------------------------------- |\n| **ABIRATERONE**   | Fatigue, joint pain/swelling, hot flashes, high blood pressure, fluid retention (edema), hypokalemia (low potassium), liver enzyme elevation.            |\n| **WYSOLONE**      | Increased appetite, fluid retention, weight gain, indigestion, mood changes, insomnia, increased blood sugar, bone density loss (with long-term use).   |\n| **INJ ZOLEDEC**   | Flu-like symptoms (fever, body aches) especially after the first

In [28]:
def gemini_prompt(prompt):
  import os
  import google.generativeai as genai
  from google.colab import userdata

  gemini_key = userdata.get('secret')

  # Configure google-generativeai library with the API key

  genai.configure(api_key = gemini_key)
  model = genai.GenerativeModel('gemini-2.5-flash')
  response = model.generate_content(prompt)
  return response

# Llama (Large Language Model Meta AI)

In [29]:
!pip install -U google-generativeai -q

import google.generativeai as genai
from IPython.display import Markdown

# Use your Google AI Studio key
genai.configure(api_key="secret")


In [30]:
!pip uninstall -y google-generativeai google-ai-generativelanguage
!pip install -U google-generativeai



Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5
Found existing installation: google-ai-generativelanguage 0.6.15
Uninstalling google-ai-generativelanguage-0.6.15:
  Successfully uninstalled google-ai-generativelanguage-0.6.15
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_generativeai-0.8.5-py3-none-any.whl (155 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)


In [31]:
import google.generativeai as genai
from IPython.display import Markdown

# Configure Gemini (replace with your API key or Colab userdata)
gen_key = userdata.get('secret')
genai.configure(api_key=gen_key)

def run_gemini(api_request_json):
    # Force correct Gemini model (v1 style)
    model_name = api_request_json.get("model", "gemini-2.5-flash")
    if not model_name.startswith("gemini"):
        model_name = "gemini-2.5-flash"

    messages = api_request_json["messages"]

    # Build prompt
    system_prompt = ""
    conversation = ""
    for msg in messages:
        if msg["role"] == "system":
            system_prompt += msg["content"] + "\n"
        elif msg["role"] == "user":
            conversation += f"User: {msg['content']}\n"

    prompt = system_prompt + "\n" + conversation

    # Use v1 API
    model = genai.GenerativeModel(model_name)
    response = model.generate_content(prompt)

    return {"choices": [{"message": {"content": response.text}}]}


# ✅ Test call
api_request_json = {
    "model": "llama3-70b",  # automatically replaced
    "messages": [
        {"role": "system", "content": "You are a llama assistant that starts every word with 'll'."},
        {"role": "user", "content": "Hi, happy llama day!"}
    ]
}

response = run_gemini(api_request_json)
print(response["choices"][0]["message"]["content"])


Llo, llo! Llucky, llovely lllama lday, llo!


In [32]:
response = run_gemini(api_request_json)
print(response["choices"][0]["message"]["content"])


Llhello! Lllovely llama lday!


In [33]:
# ✅ Prescription test using Gemini (replaces llama.run)
from IPython.display import Markdown, display

# Build API-style request (same as before)
api_request_json = {
  "model": "gemini-2.5-flash",  # replaced llama3-70b with Gemini model
  "messages": [
    {"role": "system", "content": "Please provide information about different medicines, their symptoms and their associated diseases from the prescription."},
    {"role": "user", "content": df.iloc[0][2]},
    {"role": "user", "content": df.iloc[1][2]},
    {"role": "user", "content": df.iloc[2][2]},
    {"role": "user", "content": df.iloc[3][2]}
  ]
}

# Run with Gemini wrapper
response = run_gemini(api_request_json)

# Extract model output
gemini_output = response["choices"][0]["message"]["content"]

# Display nicely
display(Markdown(gemini_output))


/tmp/ipython-input-2717343502.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  {"role": "user", "content": df.iloc[0][2]},
/tmp/ipython-input-2717343502.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  {"role": "user", "content": df.iloc[1][2]},
/tmp/ipython-input-2717343502.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  {"role": "user", "content": df.iloc[2][2]},
/tmp/ipython-input-2717343502.py:12: FutureWarning: Seri

Here's a breakdown of the medicines, their likely associated conditions (diseases), and common/important potential side effects (symptoms) based on the provided prescriptions:

---

### Prescription 1 (Antenatal Progress)

**Patient Context:** 34-year-old female, pregnant (Antenatal Progress), mentioning heartburn/acidity.

1.  **Medicine:** **VIT D3 60k every week**
    *   **Associated Disease/Use:** Vitamin D deficiency, supporting bone health for mother and baby during pregnancy.
    *   **Symptoms (Side Effects, if excessive/rare):** Nausea, vomiting, constipation, weakness, loss of appetite, excessive thirst, increased urination (symptoms of hypercalcemia, rare with prescribed dose unless pre-existing issues). Generally well-tolerated at appropriate doses.

2.  **Medicine:** **Tab. CALCIMAX once daily**
    *   **Associated Disease/Use:** Calcium deficiency, supporting bone health and development during pregnancy. CALCIMAX is typically a calcium and vitamin D supplement.
    *   **Symptoms (Side Effects):** Constipation, bloating, gas, nausea. High doses or prolonged use could rarely lead to hypercalcemia (high calcium levels) or kidney stones.

3.  **Medicine:** **Tab. FUR XT once daily**
    *   **Associated Disease/Use:** Iron deficiency anemia, folic acid supplementation. Both are crucial during pregnancy for maternal health and fetal development (preventing neural tube defects). FUR XT commonly contains ferrous fumarate (iron) and folic acid.
    *   **Symptoms (Side Effects):** Constipation, nausea, dark stools, stomach upset, diarrhea.

4.  **Medicine:** **Syp. DIGENE / MUCAINE gel 1-2 tsf SOS for heartburn / acidity**
    *   **Associated Disease/Use:** Heartburn, acidity, indigestion (common gastrointestinal discomforts during pregnancy due to hormonal changes and uterine pressure). "SOS" means "as needed." These are antacids.
    *   **Symptoms (Side Effects):** Diarrhea (if magnesium-containing), constipation (if aluminum-containing), chalky taste, nausea.

---

### Prescription 2 (Spine & Pelvis Imaging)

**Patient Context:** No explicit disease mentioned, but "CEMRI SPINE WITH PELVIS & SCREENING OF WHOLE SPINE," "BIOPSY BLOCKS," and the medications strongly suggest investigation and management of **pain** related to a spinal condition, potentially involving nerve compression or other structural issues.

1.  **Medicine:** **TAB. DOLO 650MG - TDS**
    *   **Associated Disease/Use:** Pain relief (e.g., musculoskeletal pain, headache), fever reduction. Dolo contains Paracetamol (Acetaminophen). Given the context, likely for general pain management.
    *   **Symptoms (Side Effects):** Generally well-tolerated. The most serious side effect is liver damage (hepatotoxicity), especially with overdose or chronic high doses. Nausea, stomach upset are rare.

2.  **Medicine:** **TAB TYDOLO 50MG TDS**
    *   **Associated Disease/Use:** Moderate to moderately severe pain. "Tydolo" often refers to a brand containing Tramadol, which is an opioid analgesic. Likely for more significant pain if Dolo alone is insufficient.
    *   **Symptoms (Side Effects):** Nausea, vomiting, dizziness, constipation, drowsiness, headache, dry mouth. Can be habit-forming.

3.  **Medicine:** **TAB. GABAPIN 100MG -- HS**
    *   **Associated Disease/Use:** Neuropathic pain (nerve pain), seizures, restless legs syndrome. Gabapin contains Gabapentin. Given the spine investigation, it is very likely prescribed for nerve-related pain (e.g., sciatica or pain from nerve compression). "HS" means at bedtime, often to mitigate drowsiness or target nighttime pain.
    *   **Symptoms (Side Effects):** Dizziness, drowsiness, fatigue, ataxia (difficulty with coordination), peripheral edema (swelling of hands/feet), weight gain.

---

### Prescription 3 (Diabetes Management)

**Patient Context:** 34-year-old female, "Diabetes Educator," "Blood sugar F," "Blood sugar PP," "SMBG" clearly indicate **Diabetes Mellitus** (Type 1 or Type 2 requiring insulin). The blood sugar readings (e.g., 132/15) suggest inadequate control.

1.  **Medicine:** **Inj Fiasp 4 units SC 5 min before breakfast, lunch, dinner**
    *   **Associated Disease/Use:** Diabetes Mellitus (Type 1 or Type 2). Fiasp is a rapid-acting insulin (Insulin Aspart) used to lower blood glucose levels, particularly after meals. "SC" means subcutaneous injection.
    *   **Symptoms (Side Effects):** **Hypoglycemia (low blood sugar)** is the most common and significant side effect (symptoms include shakiness, sweating, confusion, dizziness, hunger, rapid heartbeat). Injection site reactions (redness, itching, swelling), weight gain.

2.  **Medicine:** **Inj Levemir 6 units at bed time**
    *   **Associated Disease/Use:** Diabetes Mellitus (Type 1 or Type 2). Levemir is a long-acting insulin (Insulin Detemir) providing basal insulin coverage to control blood sugar levels between meals and overnight.
    *   **Symptoms (Side Effects):** **Hypoglycemia (low blood sugar)** (less common than rapid-acting insulin but can occur), injection site reactions, weight gain.

---

### Prescription 4 (Oncology Treatment)

**Patient Context:** "Medical Oncology," "Dr. Arpit Jain Consultant-III, Medical Oncology," and the specific medications strongly indicate treatment for **Cancer**, most likely **Prostate Cancer** given the prescription of Abiraterone. "iCa2+/Vitamin D3" monitoring suggests attention to bone health, which is often affected in advanced prostate cancer.

1.  **Medicine:** **TAB ABIRATERONE 1 GM OD EMPTY STOMACH**
    *   **Associated Disease/Use:** Metastatic castration-resistant prostate cancer (a form of advanced prostate cancer). Abiraterone Acetate is an androgen biosynthesis inhibitor, meaning it reduces the production of androgens (male hormones) that fuel prostate cancer growth. "OD" means once daily.
    *   **Symptoms (Side Effects):** Fatigue, joint pain/swelling, edema (fluid retention), hot flashes, diarrhea, high blood pressure, hypokalemia (low potassium), liver function abnormalities, elevated triglycerides.

2.  **Medicine:** **WYSOLONE 5 MG BD WITH MEAL**
    *   **Associated Disease/Use:** Often prescribed *with* Abiraterone to manage some of its side effects (specifically mineralocorticoid excess like hypertension and hypokalemia) and potentially for its anti-inflammatory effects or appetite stimulation. Wysolone contains Prednisolone, a corticosteroid. "BD" means twice daily.
    *   **Symptoms (Side Effects, common at this dose/duration):** Increased appetite, indigestion, insomnia, mood changes. Longer-term or higher doses can lead to more significant side effects like immunosuppression, osteoporosis, weight gain, etc.

3.  **Medicine:** **INJ ZOLEDEC RAZ 4 MG IN 100 ML NS IV OVER 30 MINS ON 15.05.24**
    *   **Associated Disease/Use:** Treatment of bone metastases (cancer spread to bones, very common in advanced prostate cancer), hypercalcemia of malignancy (high calcium due to cancer), and prevention of skeletal-related events (e.g., fractures). Zoledec RAZ likely refers to Zoledronic Acid, a bisphosphonate. "IV" means intravenous.
    *   **Symptoms (Side Effects):** Flu-like symptoms (fever, body aches, chills), bone/muscle/joint pain, fatigue, nausea, headache. More serious but rare side effects include osteonecrosis of the jaw (ONJ) and kidney dysfunction.

4.  **Medicine:** **TAB. CALCIUM 1 OD DAILY**
    *   **Associated Disease/Use:** Bone health support, especially important when receiving bisphosphonate therapy (like Zoledronic Acid) or to prevent hypocalcemia.
    *   **Symptoms (Side Effects):** Constipation, gas, bloating.

In [34]:
# Accuracy and Validity Check for Prescriptions
from IPython.display import Markdown, display

for i in range(df.shape[0]):
    prescription_text = df.iloc[i][3]  # complete text
    prompt = f"""
    Evaluate the following prescription analysis for accuracy and validity.
    Prescription: {prescription_text}

    Please provide:
    1. A validity percentage (how complete and correct is the extracted medicine/disease/food info).
    2. An error percentage (how much information is missing or incorrect).
    3. Overall accuracy estimate (based on medical correctness and completeness).
    Provide the results as a simple table:
    | Validity (%) | Error (%) | Accuracy (%) |
    """
    response = gemini_prompt(prompt)
    print(f"Prescription: {df.iloc[i][0]}")
    display(Markdown(response.text))


/tmp/ipython-input-3785382535.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prescription_text = df.iloc[i][3]  # complete text


Prescription: prescription 6.pdf


/tmp/ipython-input-3785382535.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Prescription: {df.iloc[i][0]}")


| Validity (%) | Error (%) | Accuracy (%) |
|---|---|---|
| 99 | 1 | 98 |

---

**Detailed Evaluation:**

**1. Validity Percentage (99%)**
This score reflects how complete and correct the extracted medicine, disease, and food information is *as presented in the prescription*.

*   **Medicine Information:** All medications (VIT D3 60k, CALCIMAX, FUR XT, DIGENE/MUCAINE gel) are clearly listed with their respective dosages, frequencies, and administration instructions (e.g., "every week," "once daily after breakfast," "two hours after lunch," "SOS for heartburn/acidity"). This information is complete and correctly specified for each medicine.
*   **Disease/Condition Information:** The primary condition (Antenatal Progress at 31 Weeks POG) is clearly stated. Key findings like BP (130/70), Weight (82.2 kg), Fundus Hgt (30-32 week), and Fetal Presentation (Breech relaxed) are noted. Symptoms requiring SOS medication (heartburn/acidity) are also mentioned. This information is present and appears to be a correct summary of the patient's status relevant to the consultation.
*   **Food Information:** Comprehensive dietary advice is provided: "Small frequent meals," "No outside food," "High Fiber Diet," "Avoid simple carbohydrates and sugar," and "Avoid caffeine/alcohol/Smoking." All these instructions are clear and medically appropriate for pregnancy.

The validity is extremely high because the prescription itself is a very well-structured and detailed document, ensuring that the relevant information in these categories is clearly and accurately presented.

**2. Error Percentage (1%)**
This score reflects the amount of information that is either missing or incorrect within the specified categories (medicine, disease, food).

*   **Incorrect Information:** There is no discernibly incorrect medical information related to medicines, diseases, or food advice within the prescription. The treatments, dietary, and lifestyle recommendations are all standard for antenatal care.
*   **Missing Information:** For a prescription summary, the information provided for medicine, disease, and food is highly comprehensive. Minor details like Gravida/Para status or specific height (for BMI calculation) might be present in a full medical record but are not typically mandatory for a prescription output and do not detract from the completeness of the medicine/disease/food details themselves. The phrasing "breech relaxed" is slightly less common than just "breech," but its meaning is clear in context. These minor points contribute to the very small error percentage.

**3. Overall Accuracy Estimate (98%)**
This estimate considers the medical correctness and overall completeness of the entire prescription as a plan of care.

*   **Medical Correctness:** The prescribed medications (Vitamin D3, Calcium, Iron with Folic Acid, Antacids) are appropriate and standard for a 31-week pregnant woman. The dietary and lifestyle advice (small frequent meals, high fiber, avoidance of certain foods/substances, exercise, fetal movement tracking, labor workshop, warning signs) is excellent and aligns with current antenatal guidelines. The BP reading of 130/70 is noted, and the advice seems to manage a generally healthy pregnancy with attention to common concerns.
*   **Completeness:** The prescription is very comprehensive. It includes patient demographics, gestation, relevant clinical findings, all prescribed treatments (medications, diet, lifestyle), clear instructions, advice on follow-up, and critical warning signs for reporting to the labor room. The contact details for the institution and doctor are also provided.
The prescription provides a medically sound and well-detailed plan for the patient's ongoing antenatal care.

Prescription: prescription 8.pdf


Due to the absence of a pre-existing "prescription analysis" to evaluate, I have interpreted the request as an instruction to perform the extraction of relevant medical information from the provided document and then to evaluate the completeness and correctness of that extraction as if it were the "analysis" in question.

Here's a summary of the extracted information categories:

**1. Patient Demographics:**
*   Name: MR. KHYALI RAM AGRAWAL
*   Age/Gender: 67 yrs / Male

**2. Diagnoses/Diseases:**
*   Prostatic Acinar Adenocarcinoma, Gleason's score 4+4=8, Group grade 4
*   Metastatic disease (evidenced by PSMA PET CT findings of residual prostate expression, left perirectal lymph node involvement, and multiple PSMA expressing bony metastases in pelvis and spine; lytic-sclerotic components in D3, L3, L4 vertebrae, right 2nd rib, bilateral acetabulum and pubic bones, and right ischium).
*   Degenerative changes in spine

**3. Symptoms/Chief Complaints (CIO):**
*   Lower back ache since 1 month
*   Pelvic pain since 1 month
*   Decreasing Right leg muscle mass since 1 month
*   Urinary retention in August 2022

**4. Procedures/Interventions:**
*   TURP (Transurethral Resection of Prostate) in NOV 2022
*   Prostate Biopsy (09/08/2022)

**5. Investigations/Imaging/Lab Results:**
*   **PSMA PET CT (04.04.2024):** Post TURP changes, residual PSMA expression in prostate, left perirectal lymph node, multiple PSMA expressing bony metastases in pelvis and spine.
*   **CECT (2/S/2022):** Prostate mildly enlarged with heterogeneous enhancement, radiotracer uptake, abutting bladder base with effaced fat planes on right, measures 3.6 x 3.8 x 4.9 cm, contiguous extension into bilateral seminal vesicles, abutting rectum posteriorly with maintained fat planes. Radiotracer avid bilateral external iliac, right internal iliac, and left perirectal nodes. Orthopedic hardware in situ in larger left femur. Degenerative changes in spine.
*   **PROSTATE BIOPSY (09/08/2022):** Acinar Adenocarcinoma, Gleason's score 4+4=8, Group grade 4.
*   **PSA Levels:**
    *   15/06/23: 0.77 ng/mL
    *   07/09/23: 2.130 ng/ml
    *   13.12.2023: 8.360 ng/ml
    *   28-1-24: 10.040 ng/ml
    *   7-2-24: 2.30 ng/ml
    *   07.03.24: 20.8 ng/ml
    *   06/04/2024: 22.60 ng/mL

**6. Medications (ADVICE):**
*   TAB. DOLO 650MG - TDS (three times a day)
*   TAB TYDOLO 50MG TDS (three times a day)
*   TAB. GABAPIN 100MG -- HS (at bedtime)

**7. Advised Future Investigations/Tests:**
*   CEMRI SPINE WITH PELVIS & SCREENING OF WHOLE SPINE
*   REVIEW WITH BIOPSY BLOCKS
*   DTPA SCAN
*   2D ECHO/ECG
*   CBC, KFT, LFT
*   MRI dorsolumbar spine with screening of whole spine, pelvis (with correlation with PSMA PET)

**8. Food Information:**
*   None explicitly mentioned.

**Evaluation of the Extraction (as the "analysis"):**

1.  **Validity (%)**: This refers to how complete and correct the extracted medicine, disease, and food information is.
    *   **Diseases:** All specified diseases (Prostatic Acinar Adenocarcinoma, Gleason score, Group Grade, metastatic spread, degenerative changes) are correctly and completely identified.
    *   **Medications:** All prescribed medications (DOLO 650MG, TYDOLO 50MG, GABAPIN 100MG) with their dosages and frequencies are correctly and completely identified.
    *   **Food:** No food information was present in the document, and this is correctly noted.
    *   **Assessment:** The extraction of these specific categories is 100% complete and correct relative to the provided text.

2.  **Error (%)**: This refers to how much information is missing or incorrect within the scope of medicine, disease, and food information.
    *   **Missing Information:** No medicine, disease, or food information present in the document was missed.
    *   **Incorrect Information:** No incorrect information for these categories was extracted.
    *   **Assessment:** There are no errors in the extraction of these specific categories.

3.  **Overall Accuracy Estimate (%)**: This provides a broader assessment based on medical correctness and completeness of all extracted information (not just medicine/disease/food).
    *   The extraction captured all critical patient demographics, chief complaints, detailed diagnostic information (pathology and imaging findings), historical procedures, comprehensive PSA trends, current medication advice, and recommended future investigations.
    *   All medical terms are correctly interpreted and transcribed.
    *   The extraction faithfully represents the content of the source document, including minor ambiguities in the original text (e.g., CECT date "2/S/2022" and the concatenated doctor's name). No extraneous or incorrect medical information was introduced.
    *   **Assessment:** The overall extraction is highly comprehensive and medically accurate.

---

| Validity (%) | Error (%) | Accuracy (%) |
| :----------: | :-------: | :----------: |
|     100      |     0     |      99      |

Prescription: prescription 5.pdf


Here's an evaluation of the provided prescription:

The prescription is generally well-structured and contains most of the necessary information for patient care, particularly concerning medication and basic lifestyle advice. However, there are several points of imprecision, inconsistency, and incompleteness that affect its overall quality and accuracy.

**Detailed Breakdown:**

**Strengths:**
*   **Clear Medication Instructions:** Dosage, frequency, route, and timing for both Fiasp and Levemir are very precise and medically appropriate for GDM management (rapid-acting insulin for meals, long-acting for basal).
*   **Clear Dietary Advice:** Specific and relevant instructions (increase vegetables, avoid eating within 2 hours of a major meal).
*   **Clear Monitoring & Follow-up:** SMBG advised, readings to be informed every 3rd day, and a 4-week review with SMBG readings are well-defined.
*   **Patient & Provider Identification:** Patient name, age, sex, and provider names/DMC numbers are clearly stated.

**Weaknesses / Errors / Missing Information:**
1.  **Date Incompleteness:** The year "2 IT" is incomplete and unclear.
2.  **Diagnosis Terminology:** "Pre GDM" (Pre-Gestational Diabetes Mellitus) is an unusual and potentially imprecise diagnosis for someone at "26 weeks of amenorrhoea" (26 weeks pregnant). Typically, this would be diagnosed as GDM (Gestational Diabetes Mellitus) or "pre-existing diabetes in pregnancy." This could lead to confusion regarding the patient's exact condition.
3.  **Blood Sugar Target Inconsistency:** There's an inconsistency between "Blood sugar F < 90 mg/dl" (Fasting) and "Pre meal < 95 mg/dl". Fasting is a pre-meal state, and typically the fasting target is equal to or stricter than a general pre-meal target. Having a higher pre-meal target than fasting is contradictory and confusing for the patient.
4.  **Vague Goal:** "Goal Set for 3 months" is extremely vague. A medically sound treatment plan should include specific, measurable goals (e.g., specific HbA1c target, weight management goal, specific blood sugar ranges consistently met).
5.  **Exercise Specificity:** "Regular exercise as advised by the physiotherapist" lacks specific details within *this* document, although it refers to another provider. For a comprehensive self-contained plan, more detail or a clear cross-reference to specific instructions would be beneficial.

---

**Evaluation Table:**

| Validity (%) | Error (%) | Accuracy (%) |
| :----------- | :-------- | :----------- |
| 90%          | 14%       | 88%          |

---

**Explanation of Percentages:**

*   **Validity (%): 90%**
    *   This refers to the completeness and correctness of the *extracted* medicine, disease, and food information *as presented in the prescription*.
    *   **Medicine:** 100% valid (all details clear, correct, and complete).
    *   **Disease:** "Pre GDM with 26 weeks of amenorrhoea". While the information is extracted exactly as written, the term "Pre GDM" is medically imprecise for a 26-week pregnancy, which reduces the validity of the diagnosis itself. (Estimated 70% validity for this specific item due to terminology).
    *   **Food info:** 100% valid (clear, correct, and complete).
    *   Average for these categories: (100 + 70 + 100) / 3 = 90%.

*   **Error (%): 14%**
    *   This reflects the amount of information that is missing or incorrect *within the entire prescription document*.
    *   Counting 5 significant errors/omissions: 1) Incomplete date (year), 2) Imprecise diagnosis terminology ("Pre GDM"), 3) Inconsistent blood sugar targets (Fasting vs. Pre-meal), 4) Vague goal, 5) Lack of specific exercise details.
    *   Considering approximately 35 distinct pieces of information present in the prescription, 5 errors account for (5/35) * 100 ≈ 14.3%.

*   **Overall Accuracy Estimate (%): 88%**
    *   This is a holistic assessment based on medical correctness and completeness.
    *   The prescription scores highly for the critical aspects of medication (correct drugs, doses, timings) and clear dietary advice, which are fundamental to managing the condition. Monitoring and follow-up are also clearly specified.
    *   However, the notable flaws (date, diagnostic terminology, blood sugar inconsistency, vague goal, and general exercise instructions) detract from its overall precision, completeness, and potential for optimal patient understanding and adherence. The core treatment plan is solid, but the execution of the documentation has several areas for improvement.

Prescription: prescription 7.pdf


To evaluate the "prescription analysis," I will first perform a comprehensive extraction of all relevant medical information from the provided prescription text. This extraction will serve as the "analysis" that is then evaluated for its accuracy and validity.

**Comprehensive Extraction (My "Analysis"):**

**1. Patient Information:**
*   **Name:** Mr. Khyali Ram Agrawal
*   **Age/Gender:** 67 years / Male

**2. Prescribing Doctor & Clinic:**
*   **Doctor:** Dr. Arpit Jain (Consultant-II, Medical Oncology)
    *   (Also listed: Dr. VINEET TALWARVARUNIARPIT, likely includes referring or consulting physicians or internal routing info, but Dr. Arpit Jain signs as the creator/consultant).
*   **Specialty:** Medical Oncology
*   **Clinic/Institution:** R.G.C.I. & R.C., Sec.-5, Rohini, Delhi
*   **DMC Reg. No.:** 73947
*   **Date/Time Created:** 30-04-2024, 02:00 pm

**3. Primary Diagnosis:**
*   **Prostatic Acinar Adenocarcinoma**, Gleason's score 4+4=8, Group grade 4 (Confirmed by Prostate Biopsy 09/08/2022).
*   **Metastatic disease:** Left perirectal lymph node involvement and multiple bony metastases in pelvis and spine (PSMA PET CT 04.04.2024).

**4. Past Medical History/Procedures:**
*   Underwent **TURP** (Transurethral Resection of the Prostate) in Nov 2022.

**5. Comorbidities:**
*   None known.

**6. Chief Complaints (C/O) & Current Symptoms:**
*   Lower back ache since 1 month
*   Pelvic pain since 1 month
*   Decreasing Right leg muscle mass since 1 month
*   Urinary retention in August 2022
*   Came for second opinion
*   Current pain: Right hip pain (taking 1 tab Ultracet occasionally).

**7. Investigations & Findings:**
*   **Prostate Biopsy (09/08/2022):** Acinar Adenocarcinoma, Gleason's score 4+4=8, Group grade 4.
*   **PSA (15/06/2023):** 0.77 ng/mL
*   **PSA (06/04/2024):** 22.60 ng/mL (Significant increase, indicative of progression/recurrence)
*   **PSMA PET CT (04.04.2024):**
    *   Post TURP changes with residual PSMA expression in the prostate.
    *   Left perirectal lymph node (PSMA expressing).
    *   Multiple PSMA expressing bony metastases in pelvis and spine.

**8. Vitals & General Condition:**
*   Vitals: Stable
*   CVS/CNS: NAD (No Abnormality Detected)

**9. Medications & Treatment Plan (ADVICE):**
*   **PANOPTIC NGS PANEL:** (Investigation for genomic sequencing - indicated for advanced cancer management)
*   **TAB ABIRATERONE 1 GM:**
    *   Dose: 1 GM
    *   Frequency: OD (Once Daily)
    *   Instructions: Empty stomach
*   **WYSOLONE 5 MG:**
    *   Dose: 5 MG
    *   Frequency: BD (Twice Daily)
    *   Instructions: With meal
*   **INJ ZOLEDEC RAZ 4 MG:** (Presumed Zoledronic Acid, brand name 'Zoledec Raz')
    *   Dose: 4 MG
    *   Route: IV (Intravenous)
    *   Diluent: In 100 ML NS (Normal Saline)
    *   Infusion Time: Over 30 MINS
    *   Schedule: On 15.05.24 (Single dose planned)
*   **TAB CARCIUM 1 OD DAILY:** (Presumed Calcium supplement, brand name 'Carcium' or typo for Calcium)
    *   Dose: 1 tablet
    *   Frequency: OD (Once Daily)
    *   Instructions: Daily
*   **ULTRACET:** (Pain medication, taken occasionally for Right Hip pain, dosage not specified in "ADVICE" but mentioned in C/O)

**10. Monitoring & Follow-up:**
*   BP Monitoring
*   Review after 15 days
*   Follow-up investigations: CBC, KFT, LFT, iCa2+, VITAMIN D3, ECG

---

**Evaluation of the Analysis (My Extraction):**

Assuming the above comprehensive extraction *is* the "prescription analysis" to be evaluated, I will assess its completeness and correctness relative to the provided text.

1.  **Validity (%) (how complete and correct is the extracted medicine/disease/food info):**
    *   **Medicine:** All prescribed medications (Abiraterone, Wysolone, Zoledec Raz, Carcium, Ultracet) and their details (dose, frequency, route, special instructions) are accurately extracted.
    *   **Disease:** The primary diagnosis (Prostatic Acinar Adenocarcinoma with Gleason's score and grade), metastatic sites (lymph node, bone), and progression indicators (PSA levels) are accurately captured.
    *   **Food info:** Specific instructions for medication intake related to food ("empty stomach" for Abiraterone, "with meal" for Wysolone) are correctly identified.
    *   **Conclusion:** All relevant medicine, disease, and food-related information has been completely and correctly extracted. Therefore, validity is 100%.

2.  **Error (%) (how much information is missing or incorrect):**
    *   There are no identifiable omissions of crucial medical information (diagnoses, symptoms, test results, medication details) from the provided text.
    *   There are no incorrect transcriptions or misinterpretations of the data. For "Zoledec Raz" and "Carcium," common medical assumptions (Zoledronic Acid, Calcium) are made, which is medically standard practice given the context of bone metastases.
    *   **Conclusion:** No significant information is missing, and no information is incorrect. Therefore, the error percentage is 0%.

3.  **Overall Accuracy Estimate (%):**
    *   This encompasses the entire analysis, including patient demographics, doctor information, chief complaints, comorbidities, past history, investigations, vitals, detailed medication plan, and follow-up instructions.
    *   All these aspects have been thoroughly and precisely extracted, reflecting the source text accurately and completely. The medical context and details are well-preserved.
    *   **Conclusion:** The overall accuracy estimate is 100%.

---

**Results Table:**

| Validity (%) | Error (%) | Accuracy (%) |
| :----------- | :-------- | :----------- |
| 100          | 0         | 100          |

# Complete Pipeline

In [51]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://security.ubuntu.com/ubuntu jammy-security/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 0s (898 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 125082 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...


In [52]:
!which pdfinfo
!which pdftoppm

/usr/bin/pdfinfo
/usr/bin/pdftoppm


In [53]:
### Complete Pipeline

def get_diet_recommendation():
    from google.colab import files
    from IPython.display import Markdown, display

    # Ask the user to upload a file
    print(" Please upload a PDF prescription file:")
    uploaded_file = files.upload()

    # Check if a file was uploaded
    if len(uploaded_file) > 0:
        file_name = next(iter(uploaded_file))
        if file_name.endswith(".pdf"):
            print("File uploaded successfully:", file_name)
        else:
            print("The uploaded file is not a PDF.")
            return
    else:
        print("No file uploaded.")
        return

    # Step 1 — OCR the uploaded PDF
    print("\n Reading Text...")
    ocr = read_prescription("/content/" + file_name)
    ocr_text = " ".join(ocr)

    # Step 2 — Build request for Gemini
    print("\n Identifying Diseases and Medicines...")
    api_request_json = {
        "model": "gemini-2.5-flash",
        "messages": [
            {"role": "system", "content": (
                "You are a medical analysis assistant. "
                "Extract medicines from the uploaded prescription, identify their associated symptoms "
                "and diseases, and provide food recommendations in a clear table format."
            )},
            {"role": "user", "content": ocr_text}
        ]
    }

    # Step 3 — Run Gemini request
    response = run_gemini(api_request_json)
    gemini_output = response["choices"][0]["message"]["content"]

    # Step 4 — Display results
    print("\n Generating Diet Recommendations...")
    display(Markdown(gemini_output))

    return


In [36]:
get_diet_recommendation()

 Please upload a PDF prescription file:


Saving prescription 5.pdf to prescription 5 (1).pdf
File uploaded successfully: prescription 5 (1).pdf

 Reading Text...

 Identifying Diseases and Medicines...

 Generating Diet Recommendations...


Here is the extracted information in a table format:

| Medicine           | Associated Symptoms/Diseases                                        | Food Recommendations                                                                                                                                                                                                                                                                                                                                                                    |
| :----------------- | :------------------------------------------------------------------ | :---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| Inj Fiasp          | Pre-gestational Diabetes Mellitus (Pre GDM) with 26 weeks of amenorrhoea (pregnancy). Used to control high blood sugar levels after meals. | - Increase vegetables with each meal. <br/> - Avoid having anything (snacks/extra food) within 2 hours of a major meal. <br/> - Diet as advised by the dietician (implied for managing blood glucose with diabetes).                                                                                                                                                                                                                                                             |
| Inj Levemir        | Pre-gestational Diabetes Mellitus (Pre GDM) with 26 weeks of amenorrhoea (pregnancy). Used to provide basal insulin and control high blood sugar levels, especially overnight. | - Increase vegetables with each meal. <br/> - Avoid having anything (snacks/extra food) within 2 hours of a major meal. <br/> - Diet as advised by the dietician (implied for managing blood glucose with diabetes). |

In [54]:
get_diet_recommendation()

 Please upload a PDF prescription file:


Saving Sample_Prescription.pdf to Sample_Prescription (4).pdf
File uploaded successfully: Sample_Prescription (4).pdf

 Reading Text...

 Identifying Diseases and Medicines...

 Generating Diet Recommendations...


Based on the provided prescription, here is the requested information in a clear table format:

| Medicine                     | Associated Symptoms/Diseases                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | Food Recommendations                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| :--------------------------- | :---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| Amoxicillin 250mg Oral Suspension | Amoxicillin is a penicillin-type antibiotic used to treat a wide range of bacterial infections. Common conditions for which it is prescribed include: <br> - **Ear infections (Otitis Media):** Inflammation or infection of the middle ear. <br> - **Nose and throat infections:** Such as strep throat, tonsillitis, or sinusitis. <br> - **Lower respiratory tract infections:** Like pneumonia or bronchitis. <br> - **Urinary tract infections (UTIs):** Infections in any part of the urinary system. <br> - **Skin and soft tissue infections:** Various bacterial infections affecting the skin or underlying tissues. <br> _Note: Amoxicillin is ineffective against viral infections (e.g., common cold, flu)._ | 1.  **Take with or without food:** Amoxicillin can be taken with or without food. However, taking it with a meal or snack may help reduce potential gastrointestinal side effects such as nausea or stomach upset. <br> 2.  **Probiotic-rich foods:** Antibiotics can disrupt the natural balance of beneficial bacteria in the gut, which can lead to diarrhea. Incorporate probiotic-rich foods like plain yogurt, kefir, sauerkraut, or other fermented foods. If using probiotic supplements, it's generally recommended to take them a few hours apart from the antibiotic dose to maximize their effectiveness. <br> 3.  **Stay hydrated:** Drink plenty of fluids (water, clear broths, diluted juices) throughout the day to prevent dehydration, especially if experiencing fever or diarrhea. <br> 4.  **Bland diet if stomach upset:** If you or the child experience nausea, vomiting, or diarrhea, opt for bland, easy-to-digest foods such as toast, rice, bananas, applesauce, or crackers. <br> 5.  **Avoid alcohol:** It is generally advisable to avoid alcohol while taking antibiotics, as it can worsen side effects like stomach upset and may potentially interfere with the body's ability to fight off infection and recover. <br> 6.  **Nutrient-dense foods:** Ensure a balanced diet rich in fruits, vegetables, lean proteins, and whole grains to support the immune system and aid in recovery from the infection.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| Amoxicillin 250mg Oral Suspension | Amoxicillin is a penicillin-type antibiotic used to treat a wide range of bacterial infections. Common conditions for which it is prescribed include: <br> - **Ear infections (Otitis Media):** Inflammation or infection of the middle ear. <br> - **Nose and throat infections:** Such as strep throat, tonsillitis, or sinusitis. <br> - **Lower respiratory tract infections:** Like pneumonia or bronchitis. <br> - **Urinary tract infections (UTIs):** Infections in any part of the urinary system. <br> - **Skin and soft tissue infections:** Various bacterial infections affecting the skin or underlying tissues. <br> _Note: Amoxicillin is ineffective against viral infections (e.g., common cold, flu)._ | 1.  **Take with or without food:** Amoxicillin can be taken with or without food. However, taking it with a meal or snack may help reduce potential gastrointestinal side effects such as nausea or stomach upset. <br> 2.  **Probiotic-rich foods:** Antibiotics can disrupt the natural balance of beneficial bacteria in the gut, which can lead to diarrhea. Incorporate probiotic-rich foods like plain yogurt, kefir, sauerkraut, or other fermented foods. If using probiotic supplements, it's generally recommended to take them a few hours apart from the antibiotic dose to maximize their effectiveness. <br> 3.  **Stay hydrated:** Drink plenty of fluids (water, clear broths, diluted juices) throughout the day to prevent dehydration, especially if experiencing fever or diarrhea. <br> 4.  **Bland diet if stomach upset:** If you or the child experience nausea, vomiting, or diarrhea, opt for bland, easy-to-digest foods such as toast, rice, bananas, applesauce, or crackers. <br> 5.  **Avoid alcohol:** It is generally advisable to avoid alcohol while taking antibiotics, as it can worsen side effects like stomach upset and may potentially interfere with the body's ability to fight off infection and recover. <br> 6.  **Nutrient-dense foods:** Ensure a balanced diet rich in fruits, vegetables, lean proteins, and whole grains to support the immune system and aid in recovery from the infection.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| Amoxicillin 250mg Oral Suspension | Amoxicillin is a penicillin-type antibiotic used to treat a wide range of bacterial infections. Common conditions for which it is prescribed include: <br> - **Ear infections (Otitis Media):** Inflammation or infection of the middle ear. <br> - **Nose and throat infections:** Such as strep throat, tonsillitis, or sinusitis. <br> - **Lower respiratory tract infections:** Like pneumonia or bronchitis. <br> - **Urinary tract infections (UTIs):** Infections in any part of the urinary system. <br> - **Skin and soft tissue infections:** Various bacterial infections affecting the skin or underlying tissues. <br> _Note: Amoxicillin is ineffective against viral infections (e.g., common cold, flu)._ | 1.  **Take with or without food:** Amoxicillin can be taken with or without food. However, taking it with a meal or snack may help reduce potential gastrointestinal side effects such as nausea or stomach upset. <br> 2.  **Probiotic-rich foods:** Antibiotics can disrupt the natural balance of beneficial bacteria in the gut, which can lead to diarrhea. Incorporate probiotic-rich foods like plain yogurt, kefir, sauerkraut, or other fermented foods. If using probiotic supplements, it's generally recommended to take them a few hours apart from the antibiotic dose to maximize their effectiveness. <br> 3.  **Stay hydrated:** Drink plenty of fluids (water, clear broths, diluted juices) throughout the day to prevent dehydration, especially if experiencing fever or diarrhea. <br> 4.  **Bland diet if stomach upset:** If you or the child experience nausea, vomiting, or diarrhea, opt for bland, easy-to-digest foods such as toast, rice, bananas, applesauce, or crackers. <br> 5.  **Avoid alcohol:** It is generally advisable to avoid alcohol while taking antibiotics, as it can worsen side effects like stomach upset and may potentially interfere with the body's ability to fight off infection and recover. <br> 6.  **Nutrient-dense foods:** Ensure a balanced diet rich in fruits, vegetables, lean proteins, and whole grains to support the immune system and aid in recovery from the infection.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |

This information is for guidance purposes only. For specific medical advice, please consult your healthcare provider.

In [ ]:
get_diet_recommendation()


 Please upload a PDF prescription file:


Saving prescription 5.pdf to prescription 5 (3).pdf
File uploaded successfully: prescription 5 (3).pdf

 Reading Text...

 Identifying Diseases and Medicines...
